# Paper deel 2 

De invloed van een smartphone ban nagaan op smartphone gebruik, smartphone multitasking en welbevinden. 

Op die manier vergelijking van pre-test (week 1) en post-test (week 2)

In [116]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta
from matplotlib import pyplot as plt
from scipy.stats import ttest_ind
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import linregress
import seaborn as sns
from tqdm import tqdm   #progressbar
tqdm.pandas()
sns.set();

## Nodige basisvariabelen: 

### algemeen: 

multitasking ability: TST_SC_RT & TST_SC_ACC

ALLE Data werd verwerkt, ook deze die niet gelinked kan worden met een Android. 

#### stap 1: alle data inlezen 

In [1]:
dir = "D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/data/Kortrijk/ppndata/TST"

In [6]:
TST1 = pd.read_csv(dir + "/1TST.csv")

In [7]:
TST1.head()

,sender,sender_type,sender_id,timestamp,meta,aantal,congruent,correct,correctResponse,duration,...,task switch,time_commit,time_end,time_render,time_run,time_show,time_switch,trial,url,waarde
0,Welkom,html.Screen,0,2019-01-14T14:17:50.632Z,"{""labjs_version"":""2018.0.3"",""labjs_build"":{""fl...",NaN,NaN,NaN,NaN,27808.914,...,NaN,28001.2,27995.200,19.265,198.4,186.286,28039.166,NaN,{},NaN
1,instructie AANTAL,html.Screen,1,2019-01-14T14:17:54.083Z,NaN,NaN,NaN,NaN,NaN,3411.334,...,NaN,31452.0,31450.500,27989.075,28002.2,28039.166,31487.251,NaN,NaN,NaN
2,fixatie,html.Screen,2_0_0,2019-01-14T14:17:54.304Z,NaN,d,NaN,NaN,NaN,200.336,...,NaN,31672.8,31670.890,31437.160,31452.8,31487.251,31687.587,NaN,NaN,d
3,Stimulus,html.Screen,2_0_1,2019-01-14T14:18:00.330Z,NaN,d,NaN,False,d,6010.313,...,NaN,37698.4,37697.900,31670.890,31673.2,31687.587,37715.320,NaN,NaN,d
4,ITI,html.Screen,2_0_2,2019-01-14T14:18:00.532Z,NaN,d,NaN,NaN,NaN,202.375,...,NaN,37901.5,37900.998,37698.623,37699.1,37715.320,37917.695,NaN,NaN,d


In [31]:
TST2 = pd.read_csv(dir + "/2TST.csv")

In [32]:
TST2.head()

,sender,sender_type,sender_id,timestamp,meta,aantal,congruent,correct,correctResponse,duration,...,task switch,time_commit,time_end,time_render,time_run,time_show,time_switch,trial,url,waarde
0,instructie AANTAL,html.Screen,1,2019-01-14T14:25:44.132Z,NaN,NaN,NaN,NaN,NaN,1723.491,...,NaN,29354.9,29353.300,27579.718,27592.100000,27629.809,29384.849,NaN,NaN,NaN
1,fixatie,html.Screen,2_0_0,2019-01-14T14:25:44.348Z,NaN,d,NaN,NaN,NaN,200.363,...,NaN,29570.3,29568.515,29351.455,29355.599999,29384.849,29585.212,NaN,NaN,k
2,Stimulus,html.Screen,2_0_1,2019-01-14T14:25:45.893Z,NaN,d,NaN,False,d,1528.188,...,NaN,31114.7,31113.400,29568.515,29570.700000,29585.212,31138.147,NaN,NaN,k
3,ITI,html.Screen,2_0_2,2019-01-14T14:25:46.100Z,NaN,d,NaN,NaN,NaN,200.377,...,NaN,31322.9,31321.827,31104.753,31117.300000,31138.147,31338.524,NaN,NaN,k
4,Trial,flow.Sequence,2_0,2019-01-14T14:25:46.101Z,NaN,d,NaN,NaN,NaN,1936.978,...,NaN,31323.6,31321.827,29351.455,29355.500000,29384.849,31355.221,NaN,NaN,k


In [9]:
TST3 = pd.read_csv(dir + "/3TST.csv")

In [10]:
TST4 = pd.read_csv(dir + "/4TST.csv")

In [11]:
TST5 = pd.read_csv(dir + "/5TST.csv")

In [33]:
TST6 = pd.read_csv(dir + "/6TST.csv")

In [34]:
TST6.head()

,sender,sender_type,sender_id,timestamp,meta,aantal,congruent,correct,correctResponse,duration,...,task switch,time_commit,time_end,time_render,time_run,time_show,time_switch,trial,url,waarde
0,instructie AANTAL,html.Screen,1,2019-01-14T14:19:21.987Z,NaN,NaN,NaN,NaN,NaN,13209.998,...,NaN,36798.4,36797.000,23536.911,23548.4,23587.002,36832.560,NaN,NaN,NaN
1,fixatie,html.Screen,2_0_0,2019-01-14T14:19:22.206Z,NaN,d,NaN,NaN,NaN,200.339,...,NaN,37018.1,37016.202,36799.147,36799.3,36832.560,37032.899,NaN,NaN,k
2,Stimulus,html.Screen,2_0_1,2019-01-14T14:19:38.866Z,NaN,d,NaN,True,d,16644.601,...,NaN,53678.7,53677.500,37016.202,37018.5,37032.899,53696.598,NaN,NaN,k
3,ITI,html.Screen,2_0_2,2019-01-14T14:19:39.069Z,NaN,d,NaN,NaN,NaN,200.475,...,NaN,53882.1,53880.376,53679.901,53681.5,53696.598,53897.073,NaN,NaN,k
4,Trial,flow.Sequence,2_0,2019-01-14T14:19:39.069Z,NaN,d,NaN,NaN,NaN,17047.816,...,NaN,53882.9,53880.376,36799.147,36799.1,36832.560,53913.770,NaN,NaN,k


In [13]:
TST8 = pd.read_csv(dir + "/8TST.csv")

In [14]:
TST9 = pd.read_csv(dir + "/9TST.csv")

In [15]:
TST11 = pd.read_csv(dir + "/11TST.csv")

In [16]:
TST12 = pd.read_csv(dir + "/12TST.csv")

In [17]:
TST14 = pd.read_csv(dir + "/14TST.csv")

In [18]:
TST15 = pd.read_csv(dir + "/15TST.csv")

In [19]:
TST16 = pd.read_csv(dir + "/16TST.csv")

In [20]:
TST17 = pd.read_csv(dir + "/17TST.csv")

In [21]:
TST19 = pd.read_csv(dir + "/19TST.csv")

In [22]:
TST20 = pd.read_csv(dir + "/20TST.csv")

In [23]:
TST21 = pd.read_csv(dir + "/21TST.csv")

In [24]:
TST22 = pd.read_csv(dir + "/22TST.csv")

In [25]:
TST23 = pd.read_csv(dir + "/23TST.csv")

In [26]:
TST24 = pd.read_csv(dir + "/24TST.csv")

In [27]:
TST25 = pd.read_csv(dir + "/25TST.csv")

In [28]:
TST26 = pd.read_csv(dir + "/26TST.csv")

In [29]:
TST27 = pd.read_csv(dir + "/27TST.csv")

In [30]:
TST29 = pd.read_csv(dir + "/29TST.csv")

Nu moeten alle nodige dingen geselecteerd worden. 

idem wordt gedaan met grote dataframe van Brugge

In [239]:
TST_brugge = pd.read_csv('D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/TST data Brugge/group_data_TST_brugge.csv')

In [240]:
TST_brugge.to_excel('TST_brugge.xlsx')

##### (1) zonder de oefentrials, en enkel de nodige kolommen. 

In [37]:
testTST1 = TST1[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST2 = TST2[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST3 = TST3[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST4 = TST4[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST5 = TST5[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST6 = TST6[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST8 = TST8[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST9 = TST9[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST11 = TST11[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST12 = TST12[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST14 = TST14[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST15 = TST15[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST16 = TST16[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST17 = TST17[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
#testTST18 = TST18[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST19 = TST19[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST20 = TST20[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST21 = TST21[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST22 = TST22[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST23 = TST23[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST24 = TST24[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST25 = TST25[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST26 = TST26[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST27 = TST27[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]
testTST29 = TST29[['sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task switch', 'correctResponse', 'response', 'correct',  'duration']][85:]

allemaal 1032 rijen? via len ()

In [ ]:
Van brugge zijn de oefentrials manueel verwijderd, nu moeten de kolommen geselecteerd worden die ons interesseren. 

In [241]:
TST_BRUGGE = pd.read_excel('C:/Users/beheerder/TST_brugge.xlsx')

In [243]:
TST_BRUGGE

,ppn,observation,sender,sender_type,sender_id,response,ended_on,time_run,time_render,time_show,...,meta_screen_width,meta_screen_height,meta_scroll_width,meta_scroll_height,meta_window_inner_width,meta_window_inner_height,meta_device_pixel_ratio,meta_labjs_build_flavor,meta_labjs_build_commit,url;;
86,?,04c4f,fixatie,html.Screen,6_0_0,NaN,timeout,154945.9,154933.471,154966.771,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA;;
87,?,04c4f,Stimulus,html.Screen,6_0_1,d,response,155153.9,155150.106,155166.756,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA;;
88,?,04c4f,ITI,html.Screen,6_0_2,NaN,timeout,158286.4,158280.428,158297.078,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA;;
89,?,04c4f,Trial,flow.Sequence,6_0,NaN,completion,154945.9,154933.471,154966.771,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA;;
90,?,04c4f,fixatie,html.Screen,6_1_0,NaN,timeout,158485.3,158497.005,158513.655,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA;;
91,?,04c4f,Stimulus,html.Screen,6_1_1,k,response,158701.5,158696.724,158713.374,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA;;
92,?,04c4f,ITI,html.Screen,6_1_2,NaN,timeout,162259.3,162259.899,162276.549,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA;;
93,?,04c4f,Trial,flow.Sequence,6_1,NaN,completion,158485.2,158497.005,158513.655,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA;;
94,?,04c4f,fixatie,html.Screen,6_2_0,NaN,timeout,162461.7,162476.637,162493.287,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA;;
95,?,04c4f,Stimulus,html.Screen,6_2_1,d,response,162680.8,162676.207,162692.857,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NA;;


In [249]:
testTSTBrugge = TST_BRUGGE[['ppn', 'sender', 'trial', 'stimulus', 'congruent', 'aantal', 'waarde', 'taak', 'task_switch', 'correct_response', 'response', 'correct',  'duration']]

In [250]:
testTSTBrugge.head()

,ppn,sender,trial,stimulus,congruent,aantal,waarde,taak,task_switch,correct_response,response,correct,duration
86,?,fixatie,NaN,77,NaN,d,k,NaN,NaN,NaN,NaN,NaN,199.985
87,?,Stimulus,NaN,77,NaN,d,k,NaN,NaN,d,d,1.0,3116.444
88,?,ITI,NaN,77,NaN,d,k,NaN,NaN,NaN,NaN,NaN,199.927
89,?,Trial,NaN,77,NaN,d,k,NaN,NaN,NaN,NaN,NaN,3513.584
90,?,fixatie,NaN,11111111,NaN,k,d,NaN,NaN,NaN,NaN,NaN,199.719


##### (2) enkel de rijen waar sender = Stimulus 

Belangrijk! via deze dataset kan de accuraatheid onderzocht worden. op "totaal"

voor het onderzoeken van de reactietijd moet je outliers bekijken en niet-correcte trials verwijderen. outliers zoeken op "correct". 

In [68]:
totaal1 = testTST1[testTST1['sender'] == 'Stimulus']

In [70]:
Totaal2 = testTST2[testTST2['sender'] == 'Stimulus']
Totaal3 = testTST3[testTST3['sender'] == 'Stimulus']
Totaal4 = testTST4[testTST4['sender'] == 'Stimulus']
Totaal5 = testTST5[testTST5['sender'] == 'Stimulus']
Totaal6 = testTST6[testTST6['sender'] == 'Stimulus']
Totaal8 = testTST8[testTST8['sender'] == 'Stimulus']
Totaal9 = testTST9[testTST9['sender'] == 'Stimulus']
Totaal11 = testTST11[testTST11['sender'] == 'Stimulus']
Totaal12 = testTST12[testTST12['sender'] == 'Stimulus']
Totaal14 = testTST14[testTST14['sender'] == 'Stimulus']
Totaal15 = testTST15[testTST15['sender'] == 'Stimulus']
Totaal16 = testTST16[testTST16['sender'] == 'Stimulus']
Totaal17 = testTST17[testTST17['sender'] == 'Stimulus']
Totaal19 = testTST19[testTST19['sender'] == 'Stimulus']
Totaal20 = testTST20[testTST20['sender'] == 'Stimulus']
Totaal21 = testTST21[testTST21['sender'] == 'Stimulus']
Totaal22 = testTST22[testTST22['sender'] == 'Stimulus']
Totaal23 = testTST23[testTST23['sender'] == 'Stimulus']
Totaal24 = testTST24[testTST24['sender'] == 'Stimulus']
Totaal25 = testTST25[testTST25['sender'] == 'Stimulus']
Totaal26 = testTST26[testTST26['sender'] == 'Stimulus']
Totaal27 = testTST27[testTST27['sender'] == 'Stimulus']
Totaal29 = testTST29[testTST29['sender'] == 'Stimulus']

In [251]:
totaalBrugge = testTSTBrugge[testTSTBrugge['sender'] == 'Stimulus']

In [252]:
totaalBrugge

,ppn,sender,trial,stimulus,congruent,aantal,waarde,taak,task_switch,correct_response,response,correct,duration
87,?,Stimulus,NaN,77,NaN,d,k,NaN,NaN,d,d,1.0,3116.444
91,?,Stimulus,NaN,11111111,NaN,k,d,NaN,NaN,k,k,1.0,3541.826
95,?,Stimulus,NaN,1111,NaN,d,d,NaN,NaN,d,d,1.0,5338.043
99,?,Stimulus,NaN,777,NaN,d,k,NaN,NaN,d,d,1.0,656.806
103,?,Stimulus,NaN,7777,NaN,d,k,NaN,NaN,d,d,1.0,639.928
107,?,Stimulus,NaN,3,NaN,d,d,NaN,NaN,d,d,1.0,4265.939
111,?,Stimulus,NaN,2222222,NaN,k,d,NaN,NaN,k,k,1.0,1752.226
115,?,Stimulus,NaN,99999999,NaN,k,k,NaN,NaN,k,k,1.0,791.561
119,?,Stimulus,NaN,1,NaN,d,d,NaN,NaN,d,d,1.0,973.350
123,?,Stimulus,NaN,77777777,NaN,k,k,NaN,NaN,k,k,1.0,745.076


In [ ]:
De onbekende cellen eruit doen van Brugge. Van de onbekende participanten. 

zijn alle datasets even lang? zou eigenlijk wel moeten hé. via len(). zou gelijk moeten zijn aan 256 trials. Dit klopt voor allemaal. 

In [253]:
totaalBrugge = totaalBrugge[totaalBrugge['ppn'] != '?']

In [254]:
totaalBrugge

,ppn,sender,trial,stimulus,congruent,aantal,waarde,taak,task_switch,correct_response,response,correct,duration
1202,42,Stimulus,NaN,444,NaN,d,d,NaN,NaN,d,d,1.0,2333.121
1206,42,Stimulus,NaN,77777777,NaN,k,k,NaN,NaN,k,k,1.0,1924.851
1210,42,Stimulus,NaN,444444444,NaN,k,d,NaN,NaN,k,d,0.0,834.995
1214,42,Stimulus,NaN,99,NaN,d,k,NaN,NaN,d,d,1.0,2421.691
1218,42,Stimulus,NaN,6,NaN,d,k,NaN,NaN,d,d,1.0,802.455
1222,42,Stimulus,NaN,333333333,NaN,k,d,NaN,NaN,k,k,1.0,1113.066
1226,42,Stimulus,NaN,1111,NaN,d,d,NaN,NaN,d,d,1.0,737.666
1230,42,Stimulus,NaN,2222,NaN,d,d,NaN,NaN,d,d,1.0,1096.211
1234,42,Stimulus,NaN,9,NaN,d,k,NaN,NaN,d,d,1.0,1073.786
1238,42,Stimulus,NaN,333,NaN,d,d,NaN,NaN,d,d,1.0,1219.422


wegschrijven om blokken te benoemen. 

Ondertussen ook gewoon simpel in aparte dataframes per participant verdelen. 

In [255]:
totaalBrugge.to_excel("totaalBrugge.xlsx")

##### (3) belangrijk dat elke Totaal.. dataset nu een participantnummer krijgt. 

In [95]:
totaal1["ppn"] = "1"

C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [97]:
Totaal2["ppn"] = "2"
Totaal3["ppn"] = "3"
Totaal4["ppn"] = "4"
Totaal5["ppn"] = "5"
Totaal6["ppn"] = "6"
Totaal8["ppn"] = "8"
Totaal9["ppn"] = "9"
Totaal11["ppn"] = "11"
Totaal12["ppn"] = "12"
Totaal14["ppn"] = "14"
Totaal15["ppn"] = "15"
Totaal16["ppn"] = "16"
Totaal17["ppn"] = "17"
Totaal19["ppn"] = "19"
Totaal20["ppn"] = "20"
Totaal21["ppn"] = "21"
Totaal22["ppn"] = "22"
Totaal23["ppn"] = "23"
Totaal24["ppn"] = "24"
Totaal25["ppn"] = "25"
Totaal26["ppn"] = "26"
Totaal27["ppn"] = "27"
Totaal29["ppn"] = "29"

C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [ ]:
Eerst worden de totaal blokken weggeschreven om de blokken te benoemen. en dan terug te schrijven. 

In [141]:
totaal1.to_excel("totaal1.xlsx")
Totaal2.to_excel("totaal2.xlsx")
Totaal3.to_excel("totaal3.xlsx")
Totaal4.to_excel("totaal4.xlsx")
Totaal5.to_excel("totaal5.xlsx")
Totaal6.to_excel("totaal6.xlsx")
Totaal8.to_excel("totaal8.xlsx")
Totaal9.to_excel("totaal9.xlsx")
Totaal11.to_excel("totaal11.xlsx")
Totaal12.to_excel("totaal12.xlsx")
Totaal14.to_excel("totaal14.xlsx")
Totaal15.to_excel("totaal15.xlsx")
Totaal16.to_excel("totaal16.xlsx")
Totaal17.to_excel("totaal17.xlsx")
Totaal19.to_excel("totaal19.xlsx")
Totaal20.to_excel("totaal20.xlsx")
Totaal21.to_excel("totaal21.xlsx")
Totaal22.to_excel("totaal22.xlsx")
Totaal23.to_excel("totaal23.xlsx")
Totaal24.to_excel("totaal24.xlsx")
Totaal25.to_excel("totaal25.xlsx")
Totaal26.to_excel("totaal26.xlsx")
Totaal27.to_excel("totaal27.xlsx")
Totaal29.to_excel("totaal29.xlsx")

In [143]:
totaal1 = pd.read_excel("C:/Users/beheerder/totaal1.xlsx")
Totaal2 = pd.read_excel("C:/Users/beheerder/totaal2.xlsx")
Totaal3 = pd.read_excel("C:/Users/beheerder/totaal3.xlsx")
Totaal4 = pd.read_excel("C:/Users/beheerder/totaal4.xlsx")
Totaal5 = pd.read_excel("C:/Users/beheerder/totaal5.xlsx")
Totaal6 = pd.read_excel("C:/Users/beheerder/totaal6.xlsx")
Totaal8 = pd.read_excel("C:/Users/beheerder/totaal8.xlsx")
Totaal9 = pd.read_excel("C:/Users/beheerder/totaal9.xlsx")
Totaal11 = pd.read_excel("C:/Users/beheerder/totaal11.xlsx")
Totaal12 = pd.read_excel("C:/Users/beheerder/totaal12.xlsx")
Totaal14 = pd.read_excel("C:/Users/beheerder/totaal14.xlsx")
Totaal15 = pd.read_excel("C:/Users/beheerder/totaal15.xlsx")
Totaal16 = pd.read_excel("C:/Users/beheerder/totaal16.xlsx")
Totaal17 = pd.read_excel("C:/Users/beheerder/totaal17.xlsx")
Totaal19 = pd.read_excel("C:/Users/beheerder/totaal19.xlsx")
Totaal20 = pd.read_excel("C:/Users/beheerder/totaal20.xlsx")
Totaal21 = pd.read_excel("C:/Users/beheerder/totaal21.xlsx")
Totaal22 = pd.read_excel("C:/Users/beheerder/totaal22.xlsx")
Totaal23 = pd.read_excel("C:/Users/beheerder/totaal23.xlsx")
Totaal24 = pd.read_excel("C:/Users/beheerder/totaal24.xlsx")
Totaal25 = pd.read_excel("C:/Users/beheerder/totaal25.xlsx")
Totaal26 = pd.read_excel("C:/Users/beheerder/totaal26.xlsx")
Totaal27 = pd.read_excel("C:/Users/beheerder/totaal27.xlsx")
Totaal29 = pd.read_excel("C:/Users/beheerder/totaal29.xlsx")

In [ ]:
Brugse leerlingen hetzelfde doen. 

In [257]:
totaal30 = pd.read_excel('D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/TST data Brugge/Brugge_30.xlsx')
totaal31 = pd.read_excel('D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/TST data Brugge/Brugge_31.xlsx')
totaal32 = pd.read_excel('D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/TST data Brugge/Brugge_32.xlsx')
totaal35 = pd.read_excel('D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/TST data Brugge/Brugge_35.xlsx')
totaal38 = pd.read_excel('D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/TST data Brugge/Brugge_38.xlsx')
totaal39 = pd.read_excel('D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/TST data Brugge/Brugge_39.xlsx')
totaal42 = pd.read_excel('D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/TST data Brugge/Brugge_42.xlsx')
totaal43 = pd.read_excel('D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/TST data Brugge/Brugge_43.xlsx')
totaal45 = pd.read_excel('D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/TST data Brugge/Brugge_45.xlsx')
totaal47 = pd.read_excel('D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/TST data Brugge/Brugge_47.xlsx')
totaal48 = pd.read_excel('D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/TST data Brugge/Brugge_48.xlsx')
totaal50 = pd.read_excel('D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/TST data Brugge/Brugge_50.xlsx')
totaal53 = pd.read_excel('D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/TST data Brugge/Brugge_53.xlsx')

In [258]:
totaal30.head()

,ppn,sender,blok,trial,stimulus,congruent,aantal,waarde,taak,task_switch,correct_response,response,correct,duration
15831,30,Stimulus,aantal,1,3333,NaN,d,d,NaN,NaN,d,d,1,5510.766
15835,30,Stimulus,aantal,2,1,NaN,d,d,NaN,NaN,d,d,1,2724.464
15839,30,Stimulus,aantal,3,8888888,NaN,k,k,NaN,NaN,k,k,1,1188.852
15843,30,Stimulus,aantal,4,444444,NaN,k,d,NaN,NaN,k,d,0,886.552
15847,30,Stimulus,aantal,5,1111,NaN,d,d,NaN,NaN,d,d,1,5105.499


##### (4) correct... is enkel met de correcte trials 

In [144]:
correct1 = totaal1[totaal1['correct'] == True]

In [145]:
correct2 = Totaal2[Totaal2['correct'] == True]

In [146]:
correct3 = Totaal3[Totaal3['correct'] == True]
correct4 = Totaal4[Totaal4['correct'] == True]
correct5 = Totaal5[Totaal5['correct'] == True]
correct6 = Totaal6[Totaal6['correct'] == True]
correct8 = Totaal8[Totaal8['correct'] == True]
correct9 = Totaal9[Totaal9['correct'] == True]
correct11 = Totaal11[Totaal11['correct'] == True]
correct12 = Totaal12[Totaal12['correct'] == True]
correct14 = Totaal14[Totaal14['correct'] == True]
correct15 = Totaal15[Totaal15['correct'] == True]
correct16 = Totaal16[Totaal16['correct'] == True]
correct17 = Totaal17[Totaal17['correct'] == True]
correct19 = Totaal19[Totaal19['correct'] == True]
correct20 = Totaal20[Totaal20['correct'] == True]
correct21 = Totaal21[Totaal21['correct'] == True]
correct22 = Totaal22[Totaal22['correct'] == True]
correct23 = Totaal23[Totaal23['correct'] == True]
correct24 = Totaal24[Totaal24['correct'] == True]
correct25 = Totaal25[Totaal25['correct'] == True]
correct26 = Totaal26[Totaal26['correct'] == True]
correct27 = Totaal27[Totaal27['correct'] == True]
correct29 = Totaal29[Totaal29['correct'] == True]

In [263]:
correct30 = totaal30[totaal30['correct'] == 1]
correct31 = totaal31[totaal31['correct'] == 1]
correct32 = totaal32[totaal32['correct'] == 1]
correct35 = totaal35[totaal35['correct'] == 1]
correct38 = totaal38[totaal38['correct'] == 1]
correct39 = totaal39[totaal39['correct'] == 1]
correct42 = totaal42[totaal42['correct'] == 1]
correct43 = totaal43[totaal43['correct'] == 1]
correct45 = totaal45[totaal45['correct'] == 1]
correct47 = totaal47[totaal47['correct'] == 1]
correct48 = totaal48[totaal48['correct'] == 1]
correct50 = totaal50[totaal50['correct'] == 1]
correct53 = totaal53[totaal53['correct'] == 1]

##### (5) use... is enkel zonder de outliers om zo de RTen betrouwbaar te kunnen berekenen. 

In [147]:
mean = np.mean(correct1['duration'])
std = np.std(correct1['duration'])

upper_bound, lower_bound = mean + 2 * std, mean - 2 * std

RT = correct1['duration']
correct1['outlier'] = (RT < upper_bound) & (RT > lower_bound)

use1 = correct1[correct1['outlier'] == True]

C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [148]:
mean = np.mean(correct2['duration'])
std = np.std(correct2['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct2['duration']
correct2['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use2 = correct2[correct2['outlier'] == True]

mean = np.mean(correct3['duration'])
std = np.std(correct3['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct3['duration']
correct3['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use3 = correct3[correct3['outlier'] == True]

mean = np.mean(correct4['duration'])
std = np.std(correct4['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct4['duration']
correct4['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use4 = correct4[correct4['outlier'] == True]

mean = np.mean(correct5['duration'])
std = np.std(correct5['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct5['duration']
correct5['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use5 = correct5[correct5['outlier'] == True]

mean = np.mean(correct6['duration'])
std = np.std(correct6['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct6['duration']
correct6['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use6 = correct6[correct6['outlier'] == True]

mean = np.mean(correct8['duration'])
std = np.std(correct8['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct8['duration']
correct8['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use8 = correct8[correct8['outlier'] == True]

mean = np.mean(correct9['duration'])
std = np.std(correct9['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct9['duration']
correct9['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use9 = correct9[correct9['outlier'] == True]

mean = np.mean(correct11['duration'])
std = np.std(correct11['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct11['duration']
correct11['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use11 = correct11[correct11['outlier'] == True]

mean = np.mean(correct12['duration'])
std = np.std(correct12['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct12['duration']
correct12['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use12 = correct12[correct12['outlier'] == True]

mean = np.mean(correct14['duration'])
std = np.std(correct14['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct14['duration']
correct14['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use14 = correct14[correct14['outlier'] == True]

mean = np.mean(correct15['duration'])
std = np.std(correct15['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct15['duration']
correct15['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use15 = correct15[correct15['outlier'] == True]

mean = np.mean(correct16['duration'])
std = np.std(correct16['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct16['duration']
correct16['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use16 = correct16[correct16['outlier'] == True]

mean = np.mean(correct17['duration'])
std = np.std(correct17['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct17['duration']
correct17['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use17 = correct17[correct17['outlier'] == True]

mean = np.mean(correct19['duration'])
std = np.std(correct19['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct19['duration']
correct19['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use19 = correct19[correct19['outlier'] == True]

mean = np.mean(correct20['duration'])
std = np.std(correct20['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct20['duration']
correct20['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use20 = correct20[correct20['outlier'] == True]

mean = np.mean(correct21['duration'])
std = np.std(correct21['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct21['duration']
correct21['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use21 = correct21[correct21['outlier'] == True]

mean = np.mean(correct22['duration'])
std = np.std(correct22['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct22['duration']
correct22['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use22 = correct22[correct22['outlier'] == True]

mean = np.mean(correct23['duration'])
std = np.std(correct23['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct23['duration']
correct23['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use23 = correct23[correct23['outlier'] == True]

mean = np.mean(correct24['duration'])
std = np.std(correct24['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct24['duration']
correct24['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use24 = correct24[correct24['outlier'] == True]

mean = np.mean(correct25['duration'])
std = np.std(correct25['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct25['duration']
correct25['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use25 = correct25[correct25['outlier'] == True]

mean = np.mean(correct26['duration'])
std = np.std(correct26['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct26['duration']
correct26['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use26 = correct26[correct26['outlier'] == True]

mean = np.mean(correct27['duration'])
std = np.std(correct27['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct27['duration']
correct27['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use27 = correct27[correct27['outlier'] == True]

mean = np.mean(correct29['duration'])
std = np.std(correct29['duration'])
upper_bound, lower_bound = mean + 2 * std, mean - 2 * std
RT = correct29['duration']
correct29['outlier'] = (RT < upper_bound) & (RT > lower_bound)
use29 = correct29[correct29['outlier'] == True]

C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.or

In [264]:
mean = np.mean(correct30['duration'])
std = np.std(correct30['duration'])

upper_bound, lower_bound = mean + 2 * std, mean - 2 * std

RT = correct30['duration']
correct30['outlier'] = (RT < upper_bound) & (RT > lower_bound)

use30 = correct30[correct30['outlier'] == True]

C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [265]:
mean = np.mean(correct31['duration'])
std = np.std(correct31['duration'])

upper_bound, lower_bound = mean + 2 * std, mean - 2 * std

RT = correct31['duration']
correct31['outlier'] = (RT < upper_bound) & (RT > lower_bound)

use31 = correct31[correct31['outlier'] == True]

C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [266]:
mean = np.mean(correct32['duration'])
std = np.std(correct32['duration'])

upper_bound, lower_bound = mean + 2 * std, mean - 2 * std

RT = correct32['duration']
correct32['outlier'] = (RT < upper_bound) & (RT > lower_bound)

use32 = correct32[correct32['outlier'] == True]

C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [267]:
mean = np.mean(correct35['duration'])
std = np.std(correct35['duration'])

upper_bound, lower_bound = mean + 2 * std, mean - 2 * std

RT = correct35['duration']
correct35['outlier'] = (RT < upper_bound) & (RT > lower_bound)

use35 = correct35[correct35['outlier'] == True]


C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [268]:
mean = np.mean(correct38['duration'])
std = np.std(correct38['duration'])

upper_bound, lower_bound = mean + 2 * std, mean - 2 * std

RT = correct38['duration']
correct38['outlier'] = (RT < upper_bound) & (RT > lower_bound)

use38 = correct38[correct38['outlier'] == True]


C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [269]:
mean = np.mean(correct39['duration'])
std = np.std(correct39['duration'])

upper_bound, lower_bound = mean + 2 * std, mean - 2 * std

RT = correct39['duration']
correct39['outlier'] = (RT < upper_bound) & (RT > lower_bound)

use39 = correct39[correct39['outlier'] == True]


C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [270]:
mean = np.mean(correct42['duration'])
std = np.std(correct42['duration'])

upper_bound, lower_bound = mean + 2 * std, mean - 2 * std

RT = correct42['duration']
correct42['outlier'] = (RT < upper_bound) & (RT > lower_bound)

use42 = correct42[correct42['outlier'] == True]


C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [271]:
mean = np.mean(correct43['duration'])
std = np.std(correct43['duration'])

upper_bound, lower_bound = mean + 2 * std, mean - 2 * std

RT = correct43['duration']
correct43['outlier'] = (RT < upper_bound) & (RT > lower_bound)

use43 = correct43[correct43['outlier'] == True]


C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [272]:
mean = np.mean(correct45['duration'])
std = np.std(correct45['duration'])

upper_bound, lower_bound = mean + 2 * std, mean - 2 * std

RT = correct45['duration']
correct45['outlier'] = (RT < upper_bound) & (RT > lower_bound)

use45 = correct45[correct45['outlier'] == True]


C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [273]:
mean = np.mean(correct47['duration'])
std = np.std(correct47['duration'])

upper_bound, lower_bound = mean + 2 * std, mean - 2 * std

RT = correct47['duration']
correct47['outlier'] = (RT < upper_bound) & (RT > lower_bound)

use47 = correct47[correct47['outlier'] == True]


C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [274]:
mean = np.mean(correct48['duration'])
std = np.std(correct48['duration'])

upper_bound, lower_bound = mean + 2 * std, mean - 2 * std

RT = correct48['duration']
correct48['outlier'] = (RT < upper_bound) & (RT > lower_bound)

use48 = correct48[correct48['outlier'] == True]


C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [275]:
mean = np.mean(correct50['duration'])
std = np.std(correct50['duration'])

upper_bound, lower_bound = mean + 2 * std, mean - 2 * std

RT = correct50['duration']
correct50['outlier'] = (RT < upper_bound) & (RT > lower_bound)

use50 = correct50[correct50['outlier'] == True]


C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [276]:
mean = np.mean(correct53['duration'])
std = np.std(correct53['duration'])

upper_bound, lower_bound = mean + 2 * std, mean - 2 * std

RT = correct53['duration']
correct53['outlier'] = (RT < upper_bound) & (RT > lower_bound)

use53 = correct53[correct53['outlier'] == True]


C:\Users\beheerder\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


##### (6) totale aantal fouten & outliers 

In [149]:
#aantal fouten 
totfout1 = len(totaal1) - len(correct1)
#aantal outliers 
outliers1 = len(correct1)-len(use1)

In [150]:
#aantal fouten 
totfout2 = len(Totaal2) - len(correct2)
#aantal outliers 
outliers2 = len(correct2)-len(use2)

In [151]:
#aantal fouten 
totfout3 = len(Totaal3) - len(correct3)
#aantal outliers 
outliers3 = len(correct3)-len(use3)

In [152]:
#aantal fouten 
totfout4 = len(Totaal4) - len(correct4)
#aantal outliers 
outliers4 = len(correct4)-len(use4)

In [153]:
#aantal fouten 
totfout5 = len(Totaal5) - len(correct5)
#aantal outliers 
outliers5 = len(correct5)-len(use5)

In [154]:
#aantal fouten 
totfout6 = len(Totaal6) - len(correct6)
#aantal outliers 
outliers6 = len(correct6)-len(use6)

In [155]:
#aantal fouten 
totfout8 = len(Totaal8) - len(correct8)
#aantal outliers 
outliers8 = len(correct8)-len(use8)

In [156]:
#aantal fouten 
totfout9 = len(Totaal9) - len(correct9)
#aantal outliers 
outliers9 = len(correct9)-len(use9)

In [157]:
#aantal fouten 
totfout11 = len(Totaal11) - len(correct11)
#aantal outliers 
outliers11 = len(correct11)-len(use11)

In [158]:
#aantal fouten 
totfout12 = len(Totaal12) - len(correct12)
#aantal outliers 
outliers12 = len(correct12)-len(use12)

In [159]:
#aantal fouten 
totfout14 = len(Totaal14) - len(correct14)
#aantal outliers 
outliers14 = len(correct14)-len(use14)

In [160]:
#aantal fouten 
totfout15 = len(Totaal15) - len(correct15)
#aantal outliers 
outliers15 = len(correct15)-len(use15)

In [161]:
#aantal fouten 
totfout16 = len(Totaal16) - len(correct16)
#aantal outliers 
outliers16 = len(correct16)-len(use16)

In [162]:
#aantal fouten 
totfout17 = len(Totaal17) - len(correct17)
#aantal outliers 
outliers17 = len(correct17)-len(use17)

In [163]:
#aantal fouten 
totfout19 = len(Totaal19) - len(correct19)
#aantal outliers 
outliers19 = len(correct19)-len(use19)

In [164]:
#aantal fouten 
totfout20 = len(Totaal20) - len(correct20)
#aantal outliers 
outliers20 = len(correct20)-len(use20)

In [165]:
#aantal fouten 
totfout21 = len(Totaal21) - len(correct21)
#aantal outliers 
outliers21 = len(correct21)-len(use21)

In [166]:
#aantal fouten 
totfout22 = len(Totaal22) - len(correct22)
#aantal outliers 
outliers22 = len(correct22)-len(use22)

In [167]:
#aantal fouten 
totfout23 = len(Totaal23) - len(correct23)
#aantal outliers 
outliers23 = len(correct23)-len(use23)

In [168]:
#aantal fouten 
totfout24 = len(Totaal24) - len(correct24)
#aantal outliers 
outliers24 = len(correct24)-len(use24)

In [169]:
#aantal fouten 
totfout25 = len(Totaal25) - len(correct25)
#aantal outliers 
outliers25 = len(correct25)-len(use25)

In [170]:
#aantal fouten 
totfout26 = len(Totaal26) - len(correct26)
#aantal outliers 
outliers26 = len(correct26)-len(use26)

In [171]:
#aantal fouten 
totfout27 = len(Totaal27) - len(correct27)
#aantal outliers 
outliers27 = len(correct27)-len(use27)

In [172]:
#aantal fouten 
totfout29 = len(Totaal29) - len(correct29)
#aantal outliers 
outliers29 = len(correct29)-len(use29)

Hetzelfde nu van Brugge

In [278]:
#aantal fouten 
totfout30 = len(totaal30) - len(correct30)
#aantal outliers 
outliers30 = len(correct30)-len(use30)


In [279]:
#aantal fouten 
totfout31 = len(totaal31) - len(correct31)
#aantal outliers 
outliers31 = len(correct31)-len(use31)


In [280]:
#aantal fouten 
totfout32 = len(totaal32) - len(correct32)
#aantal outliers 
outliers32 = len(correct32)-len(use32)


In [281]:
#aantal fouten 
totfout35 = len(totaal35) - len(correct35)
#aantal outliers 
outliers35 = len(correct35)-len(use35)


In [282]:
#aantal fouten 
totfout38 = len(totaal38) - len(correct38)
#aantal outliers 
outliers38 = len(correct38)-len(use38)


In [283]:
#aantal fouten 
totfout39 = len(totaal39) - len(correct39)
#aantal outliers 
outliers39 = len(correct39)-len(use39)


In [284]:
#aantal fouten 
totfout42 = len(totaal42) - len(correct42)
#aantal outliers 
outliers42 = len(correct42)-len(use42)


In [285]:
#aantal fouten 
totfout43 = len(totaal43) - len(correct43)
#aantal outliers 
outliers43 = len(correct43)-len(use43)


In [286]:
#aantal fouten 
totfout45 = len(totaal45) - len(correct45)
#aantal outliers 
outliers45 = len(correct45)-len(use45)


In [287]:
#aantal fouten 
totfout47 = len(totaal47) - len(correct47)
#aantal outliers 
outliers47 = len(correct47)-len(use47)


In [288]:
#aantal fouten 
totfout48 = len(totaal48) - len(correct48)
#aantal outliers 
outliers48 = len(correct48)-len(use48)


In [289]:
#aantal fouten 
totfout50 = len(totaal50) - len(correct50)
#aantal outliers 
outliers50 = len(correct50)-len(use50)


In [290]:
#aantal fouten 
totfout53 = len(totaal53) - len(correct53)
#aantal outliers 
outliers53 = len(correct53)-len(use53)


dit zijn dus al twee kolommen, namelijk totale aantal outliers en totale aantal fouten. 

##### (7) accuraatheid op basis van totaal..., belangrijk om dus de accuraatheidproportie van A, W, en M te hebben. Daarna de gemiddelde accuraatheid van A en W via NM en dan daarvan de proportie nemen. 

In [174]:
totaal1.head()

,sender,blok,trial,stimulus,congruent,aantal,waarde,taak,task switch,correctResponse,response,correct,duration,ppn
87,Stimulus,aantal,1,6666666,NaN,k,k,NaN,NaN,k,k,True,3531.151,1
91,Stimulus,aantal,2,777,NaN,d,k,NaN,NaN,d,d,True,964.594,1
95,Stimulus,aantal,3,777777777,NaN,k,k,NaN,NaN,k,k,True,834.303,1
99,Stimulus,aantal,4,1111,NaN,d,d,NaN,NaN,d,d,True,836.917,1
103,Stimulus,aantal,5,999999999,NaN,k,k,NaN,NaN,k,k,True,655.495,1


In [175]:
#per participant de blokken 
totaal1_A = totaal1[totaal1['blok'] == 'aantal']
totaal1_W = totaal1[totaal1['blok'] == 'waarde']
totaal1_M = totaal1[totaal1['blok'] == 'mixed']

In [183]:
totaal1_A.head()

,sender,blok,trial,stimulus,congruent,aantal,waarde,taak,task switch,correctResponse,response,correct,duration,ppn
87,Stimulus,aantal,1,6666666,NaN,k,k,NaN,NaN,k,k,True,3531.151,1
91,Stimulus,aantal,2,777,NaN,d,k,NaN,NaN,d,d,True,964.594,1
95,Stimulus,aantal,3,777777777,NaN,k,k,NaN,NaN,k,k,True,834.303,1
99,Stimulus,aantal,4,1111,NaN,d,d,NaN,NaN,d,d,True,836.917,1
103,Stimulus,aantal,5,999999999,NaN,k,k,NaN,NaN,k,k,True,655.495,1


In [190]:
totaal1_A_acc = len(totaal1_A[totaal1_A['correct'] == True])/len(totaal1_A)
totaal1_W_acc = len(totaal1_W[totaal1_W['correct'] == True])/len(totaal1_W)
totaal1_M_acc = len(totaal1_M[totaal1_M['correct'] == True])/len(totaal1_M)

In [179]:
totaal2_A = Totaal2[Totaal2['blok'] == 'aantal']
totaal2_W = Totaal2[Totaal2['blok'] == 'waarde']
totaal2_M = Totaal2[Totaal2['blok'] == 'mixed']

totaal3_A = Totaal3[Totaal3['blok'] == 'aantal']
totaal3_W = Totaal3[Totaal3['blok'] == 'waarde']
totaal3_M = Totaal3[Totaal3['blok'] == 'mixed']

totaal4_A = Totaal4[Totaal4['blok'] == 'aantal']
totaal4_W = Totaal4[Totaal4['blok'] == 'waarde']
totaal4_M = Totaal4[Totaal4['blok'] == 'mixed']

totaal5_A = Totaal5[Totaal5['blok'] == 'aantal']
totaal5_W = Totaal5[Totaal5['blok'] == 'waarde']
totaal5_M = Totaal5[Totaal5['blok'] == 'mixed']

totaal6_A = Totaal6[Totaal6['blok'] == 'aantal']
totaal6_W = Totaal6[Totaal6['blok'] == 'waarde']
totaal6_M = Totaal6[Totaal6['blok'] == 'mixed']

totaal8_A = Totaal8[Totaal8['blok'] == 'aantal']
totaal8_W = Totaal8[Totaal8['blok'] == 'waarde']
totaal8_M = Totaal8[Totaal8['blok'] == 'mixed']

totaal9_A = Totaal9[Totaal9['blok'] == 'aantal']
totaal9_W = Totaal9[Totaal9['blok'] == 'waarde']
totaal9_M = Totaal9[Totaal9['blok'] == 'mixed']

totaal11_A = Totaal11[Totaal11['blok'] == 'aantal']
totaal11_W = Totaal11[Totaal11['blok'] == 'waarde']
totaal11_M = Totaal11[Totaal11['blok'] == 'mixed']

totaal12_A = Totaal12[Totaal12['blok'] == 'aantal']
totaal12_W = Totaal12[Totaal12['blok'] == 'waarde']
totaal12_M = Totaal12[Totaal12['blok'] == 'mixed']

totaal14_A = Totaal14[Totaal14['blok'] == 'aantal']
totaal14_W = Totaal14[Totaal14['blok'] == 'waarde']
totaal14_M = Totaal14[Totaal14['blok'] == 'mixed']

totaal15_A = Totaal15[Totaal15['blok'] == 'aantal']
totaal15_W = Totaal15[Totaal15['blok'] == 'waarde']
totaal15_M = Totaal15[Totaal15['blok'] == 'mixed']

totaal16_A = Totaal16[Totaal16['blok'] == 'aantal']
totaal16_W = Totaal16[Totaal16['blok'] == 'waarde']
totaal16_M = Totaal16[Totaal16['blok'] == 'mixed']

totaal17_A = Totaal17[Totaal17['blok'] == 'aantal']
totaal17_W = Totaal17[Totaal17['blok'] == 'waarde']
totaal17_M = Totaal17[Totaal17['blok'] == 'mixed']

totaal19_A = Totaal19[Totaal19['blok'] == 'aantal']
totaal19_W = Totaal19[Totaal19['blok'] == 'waarde']
totaal19_M = Totaal19[Totaal19['blok'] == 'mixed']

totaal20_A = Totaal20[Totaal20['blok'] == 'aantal']
totaal20_W = Totaal20[Totaal20['blok'] == 'waarde']
totaal20_M = Totaal20[Totaal20['blok'] == 'mixed']

totaal21_A = Totaal21[Totaal21['blok'] == 'aantal']
totaal21_W = Totaal21[Totaal21['blok'] == 'waarde']
totaal21_M = Totaal21[Totaal21['blok'] == 'mixed']

totaal22_A = Totaal22[Totaal22['blok'] == 'aantal']
totaal22_W = Totaal22[Totaal22['blok'] == 'waarde']
totaal22_M = Totaal22[Totaal22['blok'] == 'mixed']

totaal23_A = Totaal23[Totaal23['blok'] == 'aantal']
totaal23_W = Totaal23[Totaal23['blok'] == 'waarde']
totaal23_M = Totaal23[Totaal23['blok'] == 'mixed']

totaal24_A = Totaal24[Totaal24['blok'] == 'aantal']
totaal24_W = Totaal24[Totaal24['blok'] == 'waarde']
totaal24_M = Totaal24[Totaal24['blok'] == 'mixed']

totaal25_A = Totaal25[Totaal25['blok'] == 'aantal']
totaal25_W = Totaal25[Totaal25['blok'] == 'waarde']
totaal25_M = Totaal25[Totaal25['blok'] == 'mixed']

totaal26_A = Totaal26[Totaal26['blok'] == 'aantal']
totaal26_W = Totaal26[Totaal26['blok'] == 'waarde']
totaal26_M = Totaal26[Totaal26['blok'] == 'mixed']

totaal27_A = Totaal27[Totaal27['blok'] == 'aantal']
totaal27_W = Totaal27[Totaal27['blok'] == 'waarde']
totaal27_M = Totaal27[Totaal27['blok'] == 'mixed']

totaal29_A = Totaal29[Totaal29['blok'] == 'aantal']
totaal29_W = Totaal29[Totaal29['blok'] == 'waarde']
totaal29_M = Totaal29[Totaal29['blok'] == 'mixed']


In [291]:
totaal30_A = totaal30[totaal30['blok'] == 'aantal']
totaal30_W = totaal30[totaal30['blok'] == 'waarde']
totaal30_M = totaal30[totaal30['blok'] == 'mixed']

totaal31_A = totaal31[totaal31['blok'] == 'aantal']
totaal31_W = totaal31[totaal31['blok'] == 'waarde']
totaal31_M = totaal31[totaal31['blok'] == 'mixed']

totaal32_A = totaal32[totaal32['blok'] == 'aantal']
totaal32_W = totaal32[totaal32['blok'] == 'waarde']
totaal32_M = totaal32[totaal32['blok'] == 'mixed']

totaal35_A = totaal35[totaal35['blok'] == 'aantal']
totaal35_W = totaal35[totaal35['blok'] == 'waarde']
totaal35_M = totaal35[totaal35['blok'] == 'mixed']

totaal38_A = totaal38[totaal38['blok'] == 'aantal']
totaal38_W = totaal38[totaal38['blok'] == 'waarde']
totaal38_M = totaal38[totaal38['blok'] == 'mixed']

totaal39_A = totaal39[totaal39['blok'] == 'aantal']
totaal39_W = totaal39[totaal39['blok'] == 'waarde']
totaal39_M = totaal39[totaal39['blok'] == 'mixed']

totaal42_A = totaal42[totaal42['blok'] == 'aantal']
totaal42_W = totaal42[totaal42['blok'] == 'waarde']
totaal42_M = totaal42[totaal42['blok'] == 'mixed']

totaal43_A = totaal43[totaal43['blok'] == 'aantal']
totaal43_W = totaal43[totaal43['blok'] == 'waarde']
totaal43_M = totaal43[totaal43['blok'] == 'mixed']

totaal45_A = totaal45[totaal45['blok'] == 'aantal']
totaal45_W = totaal45[totaal45['blok'] == 'waarde']
totaal45_M = totaal45[totaal45['blok'] == 'mixed']

totaal47_A = totaal47[totaal47['blok'] == 'aantal']
totaal47_W = totaal47[totaal47['blok'] == 'waarde']
totaal47_M = totaal47[totaal47['blok'] == 'mixed']

totaal48_A = totaal48[totaal48['blok'] == 'aantal']
totaal48_W = totaal48[totaal48['blok'] == 'waarde']
totaal48_M = totaal48[totaal48['blok'] == 'mixed']

totaal50_A = totaal50[totaal50['blok'] == 'aantal']
totaal50_W = totaal50[totaal50['blok'] == 'waarde']
totaal50_M = totaal50[totaal50['blok'] == 'mixed']

totaal53_A = totaal53[totaal53['blok'] == 'aantal']
totaal53_W = totaal53[totaal53['blok'] == 'waarde']
totaal53_M = totaal53[totaal53['blok'] == 'mixed']


In [191]:
totaal1_A_acc = len(totaal1_A[totaal1_A['correct'] == True])/len(totaal1_A)
totaal1_W_acc = len(totaal1_W[totaal1_W['correct'] == True])/len(totaal1_W)
totaal1_M_acc = len(totaal1_M[totaal1_M['correct'] == True])/len(totaal1_M)

totaal2_A_acc = len(totaal2_A[totaal2_A['correct'] == True])/len(totaal2_A)
totaal2_W_acc = len(totaal2_W[totaal2_W['correct'] == True])/len(totaal2_W)
totaal2_M_acc = len(totaal2_M[totaal2_M['correct'] == True])/len(totaal2_M)

totaal3_A_acc = len(totaal3_A[totaal3_A['correct'] == True])/len(totaal3_A)
totaal3_W_acc = len(totaal3_W[totaal3_W['correct'] == True])/len(totaal3_W)
totaal3_M_acc = len(totaal3_M[totaal3_M['correct'] == True])/len(totaal3_M)

totaal4_A_acc = len(totaal4_A[totaal4_A['correct'] == True])/len(totaal4_A)
totaal4_W_acc = len(totaal4_W[totaal4_W['correct'] == True])/len(totaal4_W)
totaal4_M_acc = len(totaal4_M[totaal4_M['correct'] == True])/len(totaal4_M)

totaal5_A_acc = len(totaal5_A[totaal5_A['correct'] == True])/len(totaal5_A)
totaal5_W_acc = len(totaal5_W[totaal5_W['correct'] == True])/len(totaal5_W)
totaal5_M_acc = len(totaal5_M[totaal5_M['correct'] == True])/len(totaal5_M)

totaal6_A_acc = len(totaal6_A[totaal6_A['correct'] == True])/len(totaal6_A)
totaal6_W_acc = len(totaal6_W[totaal6_W['correct'] == True])/len(totaal6_W)
totaal6_M_acc = len(totaal6_M[totaal6_M['correct'] == True])/len(totaal6_M)

totaal8_A_acc = len(totaal8_A[totaal8_A['correct'] == True])/len(totaal8_A)
totaal8_W_acc = len(totaal8_W[totaal8_W['correct'] == True])/len(totaal8_W)
totaal8_M_acc = len(totaal8_M[totaal8_M['correct'] == True])/len(totaal8_M)

totaal9_A_acc = len(totaal9_A[totaal9_A['correct'] == True])/len(totaal9_A)
totaal9_W_acc = len(totaal9_W[totaal9_W['correct'] == True])/len(totaal9_W)
totaal9_M_acc = len(totaal9_M[totaal9_M['correct'] == True])/len(totaal9_M)
totaal11_A_acc = len(totaal11_A[totaal11_A['correct'] == True])/len(totaal11_A)
totaal11_W_acc = len(totaal11_W[totaal11_W['correct'] == True])/len(totaal11_W)
totaal11_M_acc = len(totaal11_M[totaal11_M['correct'] == True])/len(totaal11_M)

totaal12_A_acc = len(totaal12_A[totaal12_A['correct'] == True])/len(totaal12_A)
totaal12_W_acc = len(totaal12_W[totaal12_W['correct'] == True])/len(totaal12_W)
totaal12_M_acc = len(totaal12_M[totaal12_M['correct'] == True])/len(totaal12_M)

totaal14_A_acc = len(totaal14_A[totaal14_A['correct'] == True])/len(totaal14_A)
totaal14_W_acc = len(totaal14_W[totaal14_W['correct'] == True])/len(totaal14_W)
totaal14_M_acc = len(totaal14_M[totaal14_M['correct'] == True])/len(totaal14_M)

totaal15_A_acc = len(totaal15_A[totaal15_A['correct'] == True])/len(totaal15_A)
totaal15_W_acc = len(totaal15_W[totaal15_W['correct'] == True])/len(totaal15_W)
totaal15_M_acc = len(totaal15_M[totaal15_M['correct'] == True])/len(totaal15_M)

totaal16_A_acc = len(totaal16_A[totaal16_A['correct'] == True])/len(totaal16_A)
totaal16_W_acc = len(totaal16_W[totaal16_W['correct'] == True])/len(totaal16_W)
totaal16_M_acc = len(totaal16_M[totaal16_M['correct'] == True])/len(totaal16_M)

totaal17_A_acc = len(totaal17_A[totaal17_A['correct'] == True])/len(totaal17_A)
totaal17_W_acc = len(totaal17_W[totaal17_W['correct'] == True])/len(totaal17_W)
totaal17_M_acc = len(totaal17_M[totaal17_M['correct'] == True])/len(totaal17_M)

totaal19_A_acc = len(totaal19_A[totaal19_A['correct'] == True])/len(totaal19_A)
totaal19_W_acc = len(totaal19_W[totaal19_W['correct'] == True])/len(totaal19_W)
totaal19_M_acc = len(totaal19_M[totaal19_M['correct'] == True])/len(totaal19_M)

totaal20_A_acc = len(totaal20_A[totaal20_A['correct'] == True])/len(totaal20_A)
totaal20_W_acc = len(totaal20_W[totaal20_W['correct'] == True])/len(totaal20_W)
totaal20_M_acc = len(totaal20_M[totaal20_M['correct'] == True])/len(totaal20_M)
totaal21_A_acc = len(totaal21_A[totaal21_A['correct'] == True])/len(totaal21_A)
totaal21_W_acc = len(totaal21_W[totaal21_W['correct'] == True])/len(totaal21_W)
totaal21_M_acc = len(totaal21_M[totaal21_M['correct'] == True])/len(totaal21_M)

totaal22_A_acc = len(totaal22_A[totaal22_A['correct'] == True])/len(totaal22_A)
totaal22_W_acc = len(totaal22_W[totaal22_W['correct'] == True])/len(totaal22_W)
totaal22_M_acc = len(totaal22_M[totaal22_M['correct'] == True])/len(totaal22_M)

totaal23_A_acc = len(totaal23_A[totaal23_A['correct'] == True])/len(totaal23_A)
totaal23_W_acc = len(totaal23_W[totaal23_W['correct'] == True])/len(totaal23_W)
totaal23_M_acc = len(totaal23_M[totaal23_M['correct'] == True])/len(totaal23_M)

totaal24_A_acc = len(totaal24_A[totaal24_A['correct'] == True])/len(totaal24_A)
totaal24_W_acc = len(totaal24_W[totaal24_W['correct'] == True])/len(totaal24_W)
totaal24_M_acc = len(totaal24_M[totaal24_M['correct'] == True])/len(totaal24_M)

totaal25_A_acc = len(totaal25_A[totaal25_A['correct'] == True])/len(totaal25_A)
totaal25_W_acc = len(totaal25_W[totaal25_W['correct'] == True])/len(totaal25_W)
totaal25_M_acc = len(totaal25_M[totaal25_M['correct'] == True])/len(totaal25_M)

totaal26_A_acc = len(totaal26_A[totaal26_A['correct'] == True])/len(totaal26_A)
totaal26_W_acc = len(totaal26_W[totaal26_W['correct'] == True])/len(totaal26_W)
totaal26_M_acc = len(totaal26_M[totaal26_M['correct'] == True])/len(totaal26_M)

totaal27_A_acc = len(totaal27_A[totaal27_A['correct'] == True])/len(totaal27_A)
totaal27_W_acc = len(totaal27_W[totaal27_W['correct'] == True])/len(totaal27_W)
totaal27_M_acc = len(totaal27_M[totaal27_M['correct'] == True])/len(totaal27_M)

totaal29_A_acc = len(totaal29_A[totaal29_A['correct'] == True])/len(totaal29_A)
totaal29_W_acc = len(totaal29_W[totaal29_W['correct'] == True])/len(totaal29_W)
totaal29_M_acc = len(totaal29_M[totaal29_M['correct'] == True])/len(totaal29_M)


In [292]:
totaal30_A_acc = len(totaal30_A[totaal30_A['correct'] == True])/len(totaal30_A)
totaal30_W_acc = len(totaal30_W[totaal30_W['correct'] == True])/len(totaal30_W)
totaal30_M_acc = len(totaal30_M[totaal30_M['correct'] == True])/len(totaal30_M)

totaal31_A_acc = len(totaal31_A[totaal31_A['correct'] == True])/len(totaal31_A)
totaal31_W_acc = len(totaal31_W[totaal31_W['correct'] == True])/len(totaal31_W)
totaal31_M_acc = len(totaal31_M[totaal31_M['correct'] == True])/len(totaal31_M)

totaal32_A_acc = len(totaal32_A[totaal32_A['correct'] == True])/len(totaal32_A)
totaal32_W_acc = len(totaal32_W[totaal32_W['correct'] == True])/len(totaal32_W)
totaal32_M_acc = len(totaal32_M[totaal32_M['correct'] == True])/len(totaal32_M)

totaal35_A_acc = len(totaal35_A[totaal35_A['correct'] == True])/len(totaal35_A)
totaal35_W_acc = len(totaal35_W[totaal35_W['correct'] == True])/len(totaal35_W)
totaal35_M_acc = len(totaal35_M[totaal35_M['correct'] == True])/len(totaal35_M)

totaal38_A_acc = len(totaal38_A[totaal38_A['correct'] == True])/len(totaal38_A)
totaal38_W_acc = len(totaal38_W[totaal38_W['correct'] == True])/len(totaal38_W)
totaal38_M_acc = len(totaal38_M[totaal38_M['correct'] == True])/len(totaal38_M)

totaal39_A_acc = len(totaal39_A[totaal39_A['correct'] == True])/len(totaal39_A)
totaal39_W_acc = len(totaal39_W[totaal39_W['correct'] == True])/len(totaal39_W)
totaal39_M_acc = len(totaal39_M[totaal39_M['correct'] == True])/len(totaal39_M)

totaal42_A_acc = len(totaal42_A[totaal42_A['correct'] == True])/len(totaal42_A)
totaal42_W_acc = len(totaal42_W[totaal42_W['correct'] == True])/len(totaal42_W)
totaal42_M_acc = len(totaal42_M[totaal42_M['correct'] == True])/len(totaal42_M)

totaal43_A_acc = len(totaal43_A[totaal43_A['correct'] == True])/len(totaal43_A)
totaal43_W_acc = len(totaal43_W[totaal43_W['correct'] == True])/len(totaal43_W)
totaal43_M_acc = len(totaal43_M[totaal43_M['correct'] == True])/len(totaal43_M)

totaal45_A_acc = len(totaal45_A[totaal45_A['correct'] == True])/len(totaal45_A)
totaal45_W_acc = len(totaal45_W[totaal45_W['correct'] == True])/len(totaal45_W)
totaal45_M_acc = len(totaal45_M[totaal45_M['correct'] == True])/len(totaal45_M)

totaal47_A_acc = len(totaal47_A[totaal47_A['correct'] == True])/len(totaal47_A)
totaal47_W_acc = len(totaal47_W[totaal47_W['correct'] == True])/len(totaal47_W)
totaal47_M_acc = len(totaal47_M[totaal47_M['correct'] == True])/len(totaal47_M)

totaal48_A_acc = len(totaal48_A[totaal48_A['correct'] == True])/len(totaal48_A)
totaal48_W_acc = len(totaal48_W[totaal48_W['correct'] == True])/len(totaal48_W)
totaal48_M_acc = len(totaal48_M[totaal48_M['correct'] == True])/len(totaal48_M)

totaal50_A_acc = len(totaal50_A[totaal50_A['correct'] == True])/len(totaal50_A)
totaal50_W_acc = len(totaal50_W[totaal50_W['correct'] == True])/len(totaal50_W)
totaal50_M_acc = len(totaal50_M[totaal50_M['correct'] == True])/len(totaal50_M)

totaal53_A_acc = len(totaal53_A[totaal53_A['correct'] == True])/len(totaal53_A)
totaal53_W_acc = len(totaal53_W[totaal53_W['correct'] == True])/len(totaal53_W)
totaal53_M_acc = len(totaal53_M[totaal53_M['correct'] == True])/len(totaal53_M)



In [ ]:
deze data in dataframe krijgen

In [220]:
# List of Tuples
Kortrijk = [ (1, totfout1, outliers1, totaal1_A_acc, totaal1_W_acc, totaal1_M_acc) ,
             (2, totfout2, outliers2, totaal2_A_acc, totaal2_W_acc, totaal2_M_acc) ,
            (3, totfout3, outliers3, totaal3_A_acc, totaal3_W_acc, totaal3_M_acc) ,
             (4, totfout4, outliers4, totaal4_A_acc, totaal4_W_acc, totaal4_M_acc),
            (5, totfout5, outliers5, totaal5_A_acc, totaal5_W_acc, totaal5_M_acc) ,
             (6, totfout6, outliers6, totaal6_A_acc, totaal6_W_acc, totaal6_M_acc),
            (8, totfout8, outliers8, totaal8_A_acc, totaal8_W_acc, totaal8_M_acc) ,
             (9, totfout9, outliers9, totaal9_A_acc, totaal9_W_acc, totaal9_M_acc),
            (11, totfout11, outliers11, totaal11_A_acc, totaal11_W_acc, totaal11_M_acc) ,
             (12, totfout12, outliers12, totaal12_A_acc, totaal12_W_acc, totaal12_M_acc),
            (14, totfout14, outliers14, totaal14_A_acc, totaal14_W_acc, totaal14_M_acc) ,
             (15, totfout15, outliers15, totaal15_A_acc, totaal15_W_acc, totaal15_M_acc),
            (16, totfout16, outliers16, totaal16_A_acc, totaal16_W_acc, totaal16_M_acc) ,
             (17, totfout17, outliers17, totaal17_A_acc, totaal17_W_acc, totaal17_M_acc),
            (19, totfout19, outliers19, totaal19_A_acc, totaal19_W_acc, totaal19_M_acc) ,
             (20, totfout20, outliers20, totaal20_A_acc, totaal20_W_acc, totaal20_M_acc),
            (21, totfout21, outliers21, totaal21_A_acc, totaal21_W_acc, totaal21_M_acc) ,
             (22, totfout22, outliers22, totaal22_A_acc, totaal22_W_acc, totaal22_M_acc),
            (23, totfout23, outliers23, totaal23_A_acc, totaal23_W_acc, totaal23_M_acc) ,
             (24, totfout24, outliers24, totaal24_A_acc, totaal24_W_acc, totaal24_M_acc),
            (25, totfout25, outliers25, totaal25_A_acc, totaal25_W_acc, totaal25_M_acc) ,
             (26, totfout26, outliers26, totaal26_A_acc, totaal26_W_acc, totaal26_M_acc) ,
            (27, totfout27, outliers27, totaal27_A_acc, totaal27_W_acc, totaal27_M_acc) ,
             (29, totfout29, outliers29, totaal29_A_acc, totaal29_W_acc, totaal29_M_acc) 
           ]
 
#Create a DataFrame object
dfObj2 = pd.DataFrame(Kortrijk, columns = ['ppn' , 'aantal_fouten', 'outliers' , 'acc_A','acc_W','acc_M']) 

In [295]:
Brugge = [ (30, totfout30, outliers30, totaal30_A_acc, totaal30_W_acc, totaal30_M_acc),
        (31, totfout31, outliers31, totaal31_A_acc, totaal31_W_acc, totaal31_M_acc),
        (32, totfout32, outliers32, totaal32_A_acc, totaal32_W_acc, totaal32_M_acc),
        (35, totfout35, outliers35, totaal35_A_acc, totaal35_W_acc, totaal35_M_acc),
        (38, totfout38, outliers38, totaal38_A_acc, totaal38_W_acc, totaal38_M_acc),
        (39, totfout39, outliers39, totaal39_A_acc, totaal39_W_acc, totaal39_M_acc),
        (42, totfout42, outliers42, totaal42_A_acc, totaal42_W_acc, totaal42_M_acc),
        (43, totfout43, outliers43, totaal43_A_acc, totaal43_W_acc, totaal43_M_acc),
        (45, totfout45, outliers45, totaal45_A_acc, totaal45_W_acc, totaal45_M_acc),
        (47, totfout47, outliers47, totaal47_A_acc, totaal47_W_acc, totaal47_M_acc),
        (48, totfout48, outliers48, totaal48_A_acc, totaal48_W_acc, totaal48_M_acc),
        (50, totfout50, outliers50, totaal50_A_acc, totaal50_W_acc, totaal50_M_acc),
        (53, totfout53, outliers53, totaal53_A_acc, totaal53_W_acc, totaal53_M_acc)]

#Create a DataFrame object
dfBr_ACC = pd.DataFrame(Brugge, columns = ['ppn' , 'aantal_fouten', 'outliers' , 'acc_A','acc_W','acc_M']) 

In [296]:
dfBr_ACC

,ppn,aantal_fouten,outliers,acc_A,acc_W,acc_M
0,30,5,10,0.984375,0.953125,0.992188
1,31,15,8,0.906250,0.968750,0.945312
2,32,5,12,1.000000,0.984375,0.968750
3,35,33,8,0.906250,0.937500,0.820312
4,38,22,6,0.968750,0.953125,0.867188
5,39,11,10,0.968750,1.000000,0.929688
6,42,4,15,0.984375,1.000000,0.976562
7,43,7,7,1.000000,1.000000,0.945312
8,45,5,8,0.984375,1.000000,0.968750
9,47,19,11,0.953125,0.843750,0.953125


In [ ]:
nu de proportie berekenen zoals dat in studie 1 gedaan is

In [ ]:
eerst NM berekenen als: acc_NM = (acc_A + acc_W)/2

In [228]:
dfObj2["acc_NM"] = (dfObj2["acc_A"]+ dfObj2["acc_W"])/2

In [297]:
dfBr_ACC["acc_NM"] = (dfBr_ACC["acc_A"]+ dfBr_ACC["acc_W"])/2

In [229]:
dfObj2.head()

,ppn,aantal_fouten,outliers,acc_A,acc_W,acc_M,acc_NM
0,1,5,8,0.984375,1.000000,0.968750,0.992188
1,2,16,11,1.000000,0.953125,0.898438,0.976562
2,3,5,10,1.000000,1.000000,0.960938,1.000000
3,4,17,8,0.953125,0.953125,0.914062,0.953125
4,5,10,19,0.921875,0.968750,0.976562,0.945312


In [ ]:
daarna de proportie van accM/accNM

In [230]:
dfObj2["prop_ACC_SC"] = dfObj2["acc_M"]/dfObj2["acc_NM"]

In [298]:
dfBr_ACC["prop_ACC_SC"] = dfBr_ACC["acc_M"]/dfBr_ACC["acc_NM"]

In [231]:
dfObj2

,ppn,aantal_fouten,outliers,acc_A,acc_W,acc_M,acc_NM,prop_ACC_SC
0,1,5,8,0.984375,1.000000,0.968750,0.992188,0.976378
1,2,16,11,1.000000,0.953125,0.898438,0.976562,0.920000
2,3,5,10,1.000000,1.000000,0.960938,1.000000,0.960938
3,4,17,8,0.953125,0.953125,0.914062,0.953125,0.959016
4,5,10,19,0.921875,0.968750,0.976562,0.945312,1.033058
5,6,11,9,0.984375,0.984375,0.929688,0.984375,0.944444
6,8,63,11,0.968750,0.546875,0.750000,0.757812,0.989691
7,9,118,9,0.968750,0.968750,0.109375,0.968750,0.112903
8,11,6,5,1.000000,0.968750,0.968750,0.984375,0.984127
9,12,73,8,0.953125,0.546875,0.679688,0.750000,0.906250


In [299]:
dfBr_ACC

,ppn,aantal_fouten,outliers,acc_A,acc_W,acc_M,acc_NM,prop_ACC_SC
0,30,5,10,0.984375,0.953125,0.992188,0.968750,1.024194
1,31,15,8,0.906250,0.968750,0.945312,0.937500,1.008333
2,32,5,12,1.000000,0.984375,0.968750,0.992188,0.976378
3,35,33,8,0.906250,0.937500,0.820312,0.921875,0.889831
4,38,22,6,0.968750,0.953125,0.867188,0.960938,0.902439
5,39,11,10,0.968750,1.000000,0.929688,0.984375,0.944444
6,42,4,15,0.984375,1.000000,0.976562,0.992188,0.984252
7,43,7,7,1.000000,1.000000,0.945312,1.000000,0.945312
8,45,5,8,0.984375,1.000000,0.968750,0.992188,0.976378
9,47,19,11,0.953125,0.843750,0.953125,0.898438,1.060870


##### (8) reactietijd op basis van use..., belangrijk om de gemiddelde reactietijd van A, W en M te hebbne en daarna de gemiddelde reactietijd van A en W via NM en dan daarvan de proportie nemen. 

In [225]:
# de drie blokken per participant
use1_A = use1[use1['blok'] == 'aantal']
use1_W = use1[use1['blok'] == 'waarde']
use1_M = use1[use1['blok'] == 'mixed']

use2_A = use2[use2['blok'] == 'aantal']
use2_W = use2[use2['blok'] == 'waarde']
use2_M = use2[use2['blok'] == 'mixed']

use3_A = use3[use3['blok'] == 'aantal']
use3_W = use3[use3['blok'] == 'waarde']
use3_M = use3[use3['blok'] == 'mixed']

use4_A = use4[use4['blok'] == 'aantal']
use4_W = use4[use4['blok'] == 'waarde']
use4_M = use4[use4['blok'] == 'mixed']

use5_A = use5[use5['blok'] == 'aantal']
use5_W = use5[use5['blok'] == 'waarde']
use5_M = use5[use5['blok'] == 'mixed']

use6_A = use6[use6['blok'] == 'aantal']
use6_W = use6[use6['blok'] == 'waarde']
use6_M = use6[use6['blok'] == 'mixed']

use8_A = use8[use8['blok'] == 'aantal']
use8_W = use8[use8['blok'] == 'waarde']
use8_M = use8[use8['blok'] == 'mixed']

use9_A = use9[use9['blok'] == 'aantal']
use9_W = use9[use9['blok'] == 'waarde']
use9_M = use9[use9['blok'] == 'mixed']

use11_A = use11[use11['blok'] == 'aantal']
use11_W = use11[use11['blok'] == 'waarde']
use11_M = use11[use11['blok'] == 'mixed']

use12_A = use12[use12['blok'] == 'aantal']
use12_W = use12[use12['blok'] == 'waarde']
use12_M = use12[use12['blok'] == 'mixed']

use14_A = use14[use14['blok'] == 'aantal']
use14_W = use14[use14['blok'] == 'waarde']
use14_M = use14[use14['blok'] == 'mixed']

use15_A = use15[use15['blok'] == 'aantal']
use15_W = use15[use15['blok'] == 'waarde']
use15_M = use15[use15['blok'] == 'mixed']

use16_A = use16[use16['blok'] == 'aantal']
use16_W = use16[use16['blok'] == 'waarde']
use16_M = use16[use16['blok'] == 'mixed']

use17_A = use17[use17['blok'] == 'aantal']
use17_W = use17[use17['blok'] == 'waarde']
use17_M = use17[use17['blok'] == 'mixed']

use19_A = use19[use19['blok'] == 'aantal']
use19_W = use19[use19['blok'] == 'waarde']
use19_M = use19[use19['blok'] == 'mixed']

use20_A = use20[use20['blok'] == 'aantal']
use20_W = use20[use20['blok'] == 'waarde']
use20_M = use20[use20['blok'] == 'mixed']

use21_A = use21[use21['blok'] == 'aantal']
use21_W = use21[use21['blok'] == 'waarde']
use21_M = use21[use21['blok'] == 'mixed']

use22_A = use22[use22['blok'] == 'aantal']
use22_W = use22[use22['blok'] == 'waarde']
use22_M = use22[use22['blok'] == 'mixed']

use23_A = use23[use23['blok'] == 'aantal']
use23_W = use23[use23['blok'] == 'waarde']
use23_M = use23[use23['blok'] == 'mixed']

use24_A = use24[use24['blok'] == 'aantal']
use24_W = use24[use24['blok'] == 'waarde']
use24_M = use24[use24['blok'] == 'mixed']

use25_A = use25[use25['blok'] == 'aantal']
use25_W = use25[use25['blok'] == 'waarde']
use25_M = use25[use25['blok'] == 'mixed']

use26_A = use26[use26['blok'] == 'aantal']
use26_W = use26[use26['blok'] == 'waarde']
use26_M = use26[use26['blok'] == 'mixed']

use27_A = use27[use27['blok'] == 'aantal']
use27_W = use27[use27['blok'] == 'waarde']
use27_M = use27[use27['blok'] == 'mixed']

use29_A = use29[use29['blok'] == 'aantal']
use29_W = use29[use29['blok'] == 'waarde']
use29_M = use29[use29['blok'] == 'mixed']


In [ ]:
idem voor Brugge

In [300]:
use30_A = use30[use30['blok'] == 'aantal']
use30_W = use30[use30['blok'] == 'waarde']
use30_M = use30[use30['blok'] == 'mixed']

use31_A = use31[use31['blok'] == 'aantal']
use31_W = use31[use31['blok'] == 'waarde']
use31_M = use31[use31['blok'] == 'mixed']

use32_A = use32[use32['blok'] == 'aantal']
use32_W = use32[use32['blok'] == 'waarde']
use32_M = use32[use32['blok'] == 'mixed']

use35_A = use35[use35['blok'] == 'aantal']
use35_W = use35[use35['blok'] == 'waarde']
use35_M = use35[use35['blok'] == 'mixed']

use38_A = use38[use38['blok'] == 'aantal']
use38_W = use38[use38['blok'] == 'waarde']
use38_M = use38[use38['blok'] == 'mixed']

use39_A = use39[use39['blok'] == 'aantal']
use39_W = use39[use39['blok'] == 'waarde']
use39_M = use39[use39['blok'] == 'mixed']

use42_A = use42[use42['blok'] == 'aantal']
use42_W = use42[use42['blok'] == 'waarde']
use42_M = use42[use42['blok'] == 'mixed']

use43_A = use43[use43['blok'] == 'aantal']
use43_W = use43[use43['blok'] == 'waarde']
use43_M = use43[use43['blok'] == 'mixed']

use45_A = use45[use45['blok'] == 'aantal']
use45_W = use45[use45['blok'] == 'waarde']
use45_M = use45[use45['blok'] == 'mixed']

use47_A = use47[use47['blok'] == 'aantal']
use47_W = use47[use47['blok'] == 'waarde']
use47_M = use47[use47['blok'] == 'mixed']

use48_A = use48[use48['blok'] == 'aantal']
use48_W = use48[use48['blok'] == 'waarde']
use48_M = use48[use48['blok'] == 'mixed']

use50_A = use50[use50['blok'] == 'aantal']
use50_W = use50[use50['blok'] == 'waarde']
use50_M = use50[use50['blok'] == 'mixed']

use53_A = use53[use53['blok'] == 'aantal']
use53_W = use53[use53['blok'] == 'waarde']
use53_M = use53[use53['blok'] == 'mixed']


Gemiddelde reactietijd per blok bekomen. 

In [226]:
meanRT1_A = use1_A['duration'].mean()
meanRT1_W = use1_W['duration'].mean()
meanRT1_M = use1_M['duration'].mean()

meanRT2_A = use2_A['duration'].mean()
meanRT2_W = use2_W['duration'].mean()
meanRT2_M = use2_M['duration'].mean()

meanRT3_A = use3_A['duration'].mean()
meanRT3_W = use3_W['duration'].mean()
meanRT3_M = use3_M['duration'].mean()

meanRT4_A = use4_A['duration'].mean()
meanRT4_W = use4_W['duration'].mean()
meanRT4_M = use4_M['duration'].mean()

meanRT5_A = use5_A['duration'].mean()
meanRT5_W = use5_W['duration'].mean()
meanRT5_M = use5_M['duration'].mean()

meanRT6_A = use6_A['duration'].mean()
meanRT6_W = use6_W['duration'].mean()
meanRT6_M = use6_M['duration'].mean()

meanRT8_A = use8_A['duration'].mean()
meanRT8_W = use8_W['duration'].mean()
meanRT8_M = use8_M['duration'].mean()

meanRT9_A = use9_A['duration'].mean()
meanRT9_W = use9_W['duration'].mean()
meanRT9_M = use9_M['duration'].mean()

meanRT11_A = use11_A['duration'].mean()
meanRT11_W = use11_W['duration'].mean()
meanRT11_M = use11_M['duration'].mean()

meanRT12_A = use12_A['duration'].mean()
meanRT12_W = use12_W['duration'].mean()
meanRT12_M = use12_M['duration'].mean()

meanRT14_A = use14_A['duration'].mean()
meanRT14_W = use14_W['duration'].mean()
meanRT14_M = use14_M['duration'].mean()

meanRT15_A = use15_A['duration'].mean()
meanRT15_W = use15_W['duration'].mean()
meanRT15_M = use15_M['duration'].mean()

meanRT16_A = use16_A['duration'].mean()
meanRT16_W = use16_W['duration'].mean()
meanRT16_M = use16_M['duration'].mean()

meanRT17_A = use17_A['duration'].mean()
meanRT17_W = use17_W['duration'].mean()
meanRT17_M = use17_M['duration'].mean()

meanRT19_A = use19_A['duration'].mean()
meanRT19_W = use19_W['duration'].mean()
meanRT19_M = use19_M['duration'].mean()

meanRT20_A = use20_A['duration'].mean()
meanRT20_W = use20_W['duration'].mean()
meanRT20_M = use20_M['duration'].mean()

meanRT21_A = use21_A['duration'].mean()
meanRT21_W = use21_W['duration'].mean()
meanRT21_M = use21_M['duration'].mean()

meanRT22_A = use22_A['duration'].mean()
meanRT22_W = use22_W['duration'].mean()
meanRT22_M = use22_M['duration'].mean()


meanRT23_A = use23_A['duration'].mean()
meanRT23_W = use23_W['duration'].mean()
meanRT23_M = use23_M['duration'].mean()


meanRT24_A = use24_A['duration'].mean()
meanRT24_W = use24_W['duration'].mean()
meanRT24_M = use24_M['duration'].mean()


meanRT25_A = use25_A['duration'].mean()
meanRT25_W = use25_W['duration'].mean()
meanRT25_M = use25_M['duration'].mean()


meanRT26_A = use26_A['duration'].mean()
meanRT26_W = use26_W['duration'].mean()
meanRT26_M = use26_M['duration'].mean()


meanRT27_A = use27_A['duration'].mean()
meanRT27_W = use27_W['duration'].mean()
meanRT27_M = use27_M['duration'].mean()


meanRT29_A = use29_A['duration'].mean()
meanRT29_W = use29_W['duration'].mean()
meanRT29_M = use29_M['duration'].mean()



In [ ]:
Idem met Brugge

In [301]:
meanRT30_A = use30_A['duration'].mean()
meanRT30_W = use30_W['duration'].mean()
meanRT30_M = use30_M['duration'].mean()

meanRT31_A = use31_A['duration'].mean()
meanRT31_W = use31_W['duration'].mean()
meanRT31_M = use31_M['duration'].mean()

meanRT32_A = use32_A['duration'].mean()
meanRT32_W = use32_W['duration'].mean()
meanRT32_M = use32_M['duration'].mean()

meanRT35_A = use35_A['duration'].mean()
meanRT35_W = use35_W['duration'].mean()
meanRT35_M = use35_M['duration'].mean()

meanRT38_A = use38_A['duration'].mean()
meanRT38_W = use38_W['duration'].mean()
meanRT38_M = use38_M['duration'].mean()

meanRT39_A = use39_A['duration'].mean()
meanRT39_W = use39_W['duration'].mean()
meanRT39_M = use39_M['duration'].mean()

meanRT42_A = use42_A['duration'].mean()
meanRT42_W = use42_W['duration'].mean()
meanRT42_M = use42_M['duration'].mean()

meanRT43_A = use43_A['duration'].mean()
meanRT43_W = use43_W['duration'].mean()
meanRT43_M = use43_M['duration'].mean()

meanRT45_A = use45_A['duration'].mean()
meanRT45_W = use45_W['duration'].mean()
meanRT45_M = use45_M['duration'].mean()

meanRT47_A = use47_A['duration'].mean()
meanRT47_W = use47_W['duration'].mean()
meanRT47_M = use47_M['duration'].mean()

meanRT48_A = use48_A['duration'].mean()
meanRT48_W = use48_W['duration'].mean()
meanRT48_M = use48_M['duration'].mean()

meanRT50_A = use50_A['duration'].mean()
meanRT50_W = use50_W['duration'].mean()
meanRT50_M = use50_M['duration'].mean()

meanRT53_A = use53_A['duration'].mean()
meanRT53_W = use53_W['duration'].mean()
meanRT53_M = use53_M['duration'].mean()


In [227]:
# List of Tuples
Kortrijk_RT = [ (1, meanRT1_A, meanRT1_W, meanRT1_M) ,
             (2, meanRT2_A, meanRT2_W, meanRT2_M) ,
            (3, meanRT3_A, meanRT3_W, meanRT3_M) ,
             (4, meanRT4_A, meanRT4_W, meanRT4_M),
            (5, meanRT5_A, meanRT5_W, meanRT5_M) ,
             (6, meanRT6_A, meanRT6_W, meanRT6_M),
            (8, meanRT8_A, meanRT8_W, meanRT8_M) ,
             (9, meanRT9_A, meanRT9_W, meanRT9_M),
            (11, meanRT11_A, meanRT11_W, meanRT11_M) ,
             (12, meanRT12_A, meanRT12_W, meanRT12_M),
            (14, meanRT14_A, meanRT14_W, meanRT14_M) ,
             (15, meanRT15_A, meanRT15_W, meanRT15_M),
            (16, meanRT16_A, meanRT16_W, meanRT16_M) ,
             (17, meanRT17_A, meanRT17_W, meanRT17_M),
            (19, meanRT19_A, meanRT19_W, meanRT19_M) ,
             (20, meanRT20_A, meanRT20_W, meanRT20_M),
            (21, meanRT21_A, meanRT21_W, meanRT21_M) ,
             (22, meanRT22_A, meanRT22_W, meanRT22_M),
            (23, meanRT23_A, meanRT23_W, meanRT23_M) ,
             (24, meanRT24_A, meanRT24_W, meanRT24_M),
            (25, meanRT25_A, meanRT25_W, meanRT25_M) ,
             (26, meanRT26_A, meanRT26_W, meanRT26_M) ,
            (27, meanRT27_A, meanRT27_W, meanRT27_M) ,
             (29, meanRT29_A, meanRT29_W, meanRT29_M) 
           ]
 
#Create a DataFrame object
dfObj2_RT = pd.DataFrame(Kortrijk_RT, columns = ['ppn','meanRT_A', 'meanRT_W', 'meanRT_M']) 

In [303]:
Brugge_RT = [ (30, meanRT30_A, meanRT30_W, meanRT30_M) ,
             (31, meanRT31_A, meanRT31_W, meanRT31_M) ,
             (32, meanRT32_A, meanRT32_W, meanRT32_M) ,
             (35, meanRT35_A, meanRT35_W, meanRT35_M) ,
             (38, meanRT38_A, meanRT38_W, meanRT38_M) ,
             (39, meanRT39_A, meanRT39_W, meanRT39_M) ,
             (42, meanRT42_A, meanRT42_W, meanRT42_M) ,
             (43, meanRT43_A, meanRT43_W, meanRT43_M) ,
             (45, meanRT45_A, meanRT45_W, meanRT45_M) ,
             (47, meanRT47_A, meanRT47_W, meanRT47_M) ,
             (48, meanRT48_A, meanRT48_W, meanRT48_M) ,
             (50, meanRT50_A, meanRT50_W, meanRT50_M) ,
             (53, meanRT53_A, meanRT53_W, meanRT53_M) ,
           ]
dfBr_RT = pd.DataFrame(Brugge_RT, columns = ['ppn','meanRT_A', 'meanRT_W', 'meanRT_M'])

In [232]:
dfObj2_RT.head()

,ppn,meanRT_A,meanRT_W,meanRT_M
0,1,1088.121825,1122.969000,1515.985872
1,2,887.252793,980.179426,1204.204533
2,3,871.876109,770.820344,1483.649155
3,4,812.436067,740.471552,1085.794982
4,5,797.144397,734.133733,899.676000


In [ ]:
eerst NM vormen 

In [233]:
dfObj2_RT["meanRT_NM"] = (dfObj2_RT["meanRT_A"] + dfObj2_RT["meanRT_W"])/2

In [304]:
dfBr_RT["meanRT_NM"] = (dfBr_RT["meanRT_A"] + dfBr_RT["meanRT_W"])/2

In [ ]:
daarna prop_RT_SC

In [234]:
dfObj2_RT["prop_RT_SC"] = dfObj2_RT["meanRT_M"]/dfObj2_RT["meanRT_NM"]

In [305]:
dfBr_RT["prop_RT_SC"] = dfBr_RT["meanRT_M"]/dfBr_RT["meanRT_NM"]

In [235]:
dfObj2_RT.head()

,ppn,meanRT_A,meanRT_W,meanRT_M,meanRT_NM,prop_RT_SC
0,1,1088.121825,1122.969000,1515.985872,1105.545413,1.371256
1,2,887.252793,980.179426,1204.204533,933.716110,1.289690
2,3,871.876109,770.820344,1483.649155,821.348227,1.806358
3,4,812.436067,740.471552,1085.794982,776.453809,1.398403
4,5,797.144397,734.133733,899.676000,765.639065,1.175065


In [306]:
dfBr_RT.head()

,ppn,meanRT_A,meanRT_W,meanRT_M,meanRT_NM,prop_RT_SC
0,30,1080.457758,1130.710966,1292.338694,1105.584362,1.168919
1,31,858.313286,832.648836,1234.966267,845.481061,1.460667
2,32,962.991705,981.964774,1009.962603,972.478240,1.038545
3,35,762.829842,677.383034,932.153293,720.106438,1.294466
4,38,731.669750,597.903049,956.540822,664.786400,1.438869


In [236]:
Kortrijk = pd.merge(dfObj2, dfObj2_RT, on = "ppn")

In [308]:
Brugge = pd.merge(dfBr_ACC, dfBr_RT, on = "ppn")

In [309]:
Brugge

,ppn,aantal_fouten,outliers,acc_A,acc_W,acc_M,acc_NM,prop_ACC_SC,meanRT_A,meanRT_W,meanRT_M,meanRT_NM,prop_RT_SC
0,30,5,10,0.984375,0.953125,0.992188,0.968750,1.024194,1080.457758,1130.710966,1292.338694,1105.584362,1.168919
1,31,15,8,0.906250,0.968750,0.945312,0.937500,1.008333,858.313286,832.648836,1234.966267,845.481061,1.460667
2,32,5,12,1.000000,0.984375,0.968750,0.992188,0.976378,962.991705,981.964774,1009.962603,972.478240,1.038545
3,35,33,8,0.906250,0.937500,0.820312,0.921875,0.889831,762.829842,677.383034,932.153293,720.106438,1.294466
4,38,22,6,0.968750,0.953125,0.867188,0.960938,0.902439,731.669750,597.903049,956.540822,664.786400,1.438869
5,39,11,10,0.968750,1.000000,0.929688,0.984375,0.944444,835.432117,679.989203,1008.218460,757.710660,1.330611
6,42,4,15,0.984375,1.000000,0.976562,0.992188,0.984252,974.497000,916.591016,1355.667425,945.544008,1.433743
7,43,7,7,1.000000,1.000000,0.945312,1.000000,0.945312,1678.820476,1622.007433,1853.837613,1650.413955,1.123256
8,45,5,8,0.984375,1.000000,0.968750,0.992188,0.976378,1041.145217,882.098000,1272.832294,961.621608,1.323631
9,47,19,11,0.953125,0.843750,0.953125,0.898438,1.060870,1000.351034,950.924692,1510.626261,975.637863,1.548347


In [238]:
Kortrijk.to_excel("Task-switching-kortrijk.xlsx")

In [310]:
Brugge.to_excel("Task-switching-brugge.xlsx")

## Data Kortrijk inlezen 

In [3]:
app_filename = "D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/MobileDNA Kortrijk/floor_2_appevents.csv"
notif_filename = "D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/MobileDNA Kortrijk/floor_2_notifications.csv"

In [117]:
app_Brugge = "D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/floor_brugge_2_appevents.csv"
notif_Brugge = "D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/floor_brugge_2_notifications.csv"

In [12]:
df = pd.read_csv(app_filename, sep=";", parse_dates=['startTime', 'endTime'])
df.drop("Unnamed: 0", axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34573 entries, 0 to 34572
Data columns (total 15 columns):
application        34573 non-null object
battery            34573 non-null int64
data_version       34573 non-null float64
endTime            34573 non-null datetime64[ns]
endTimeMillis      34573 non-null float64
id                 34573 non-null object
latitude           34573 non-null float64
longitude          34573 non-null float64
model              34573 non-null object
notification       34573 non-null bool
session            34573 non-null int64
startTime          34573 non-null datetime64[ns]
startTimeMillis    34573 non-null float64
studyKey           34573 non-null object
surveyId           34573 non-null object
dtypes: bool(1), datetime64[ns](2), float64(5), int64(2), object(5)
memory usage: 3.7+ MB


In [118]:
dfBr = pd.read_csv(app_Brugge, sep=";", parse_dates=['startTime', 'endTime'])
dfBr.drop("Unnamed: 0", axis=1, inplace=True)
dfBr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13152 entries, 0 to 13151
Data columns (total 15 columns):
application        13152 non-null object
battery            13152 non-null float64
data_version       13152 non-null float64
endTime            13152 non-null datetime64[ns]
endTimeMillis      13152 non-null int64
id                 13152 non-null object
latitude           13152 non-null float64
longitude          13152 non-null float64
model              13152 non-null object
notification       13152 non-null bool
session            13152 non-null int64
startTime          13152 non-null datetime64[ns]
startTimeMillis    13152 non-null int64
studyKey           13152 non-null object
surveyId           13152 non-null object
dtypes: bool(1), datetime64[ns](2), float64(4), int64(3), object(5)
memory usage: 1.4+ MB


In [119]:
#checken voor dubbels 
dubbels = pd.concat(g for _, g in df.groupby(["application","startTimeMillis","endTime"]) if len(g) > 1)

dubbels['session'].value_counts().describe()

count    101.000000
mean      10.376238
std       18.034884
min        2.000000
25%        2.000000
50%        4.000000
75%        8.000000
max      114.000000
Name: session, dtype: float64

In [120]:
#checken voor dubbels 
dubbelsBr = pd.concat(g for _, g in dfBr.groupby(["application","startTimeMillis","endTime"]) if len(g) > 1)

dubbelsBr['session'].value_counts().describe()

count    87.000000
mean      6.367816
std       9.733800
min       2.000000
25%       2.000000
50%       4.000000
75%       6.000000
max      54.000000
Name: session, dtype: float64

In [6]:
df.session.nunique()

8320

In [121]:
dfBr.session.nunique()

4147

In [7]:
df.head(5)

,application,battery,data_version,endTime,endTimeMillis,id,latitude,longitude,model,notification,session,startTime,startTimeMillis,studyKey,surveyId
0,com.snapchat.android,67,1.5,2019-01-17 16:32:16.801,1.547739e+12,a29814bf-bd90-48e6-b42e-368285fd7579,0.0,0.0,MYA-L41,False,1547738718,2019-01-17 16:31:56.753,1.547739e+12,_,_
1,com.snapchat.android,61,1.5,2019-01-17 17:01:53.643,1.547741e+12,a29814bf-bd90-48e6-b42e-368285fd7579,0.0,0.0,MYA-L41,False,1547740566,2019-01-17 17:00:34.860,1.547741e+12,_,_
2,com.twitter.android,61,1.5,2019-01-17 17:15:23.889,1.547742e+12,a29814bf-bd90-48e6-b42e-368285fd7579,0.0,0.0,MYA-L41,False,1547740566,2019-01-17 17:05:48.295,1.547741e+12,_,_
3,com.snapchat.android,46,1.5,2019-01-17 17:56:24.387,1.547744e+12,a29814bf-bd90-48e6-b42e-368285fd7579,0.0,0.0,MYA-L41,False,1547741927,2019-01-17 17:56:20.227,1.547744e+12,_,_
4,com.android.mms,42,1.5,2019-01-17 18:32:38.196,1.547746e+12,a29814bf-bd90-48e6-b42e-368285fd7579,0.0,0.0,MYA-L41,False,1547745578,2019-01-17 18:32:11.967,1.547746e+12,_,_


In [122]:
dfBr.head()

,application,battery,data_version,endTime,endTimeMillis,id,latitude,longitude,model,notification,session,startTime,startTimeMillis,studyKey,surveyId
0,com.android.systemui,29.0,1.5,2019-01-23 19:07:26.649,1548266846649,00805caf-ff20-4bfb-964f-8bc01b035569,51.198560,3.240879,SM-A600FN,False,1548266758,2019-01-23 19:07:24.222,1548266844222,_,_
1,com.android.chrome,26.0,1.5,2019-01-23 19:14:53.040,1548267293040,00805caf-ff20-4bfb-964f-8bc01b035569,51.198560,3.240879,SM-A600FN,False,1548267175,2019-01-23 19:14:37.246,1548267277246,_,_
2,com.facebook.orca,89.0,1.5,2019-01-23 14:44:38.098,1548251078098,00805caf-ff20-4bfb-964f-8bc01b035569,51.215329,3.249471,SM-A600FN,False,1548251027,2019-01-23 14:43:57.623,1548251037623,_,_
3,com.sec.android.app.sbrowser,89.0,1.5,2019-01-23 14:45:20.986,1548251120986,00805caf-ff20-4bfb-964f-8bc01b035569,51.215329,3.249471,SM-A600FN,False,1548251027,2019-01-23 14:45:20.968,1548251120968,_,_
4,com.sec.android.app.sbrowser,89.0,1.5,2019-01-23 14:45:22.545,1548251122545,00805caf-ff20-4bfb-964f-8bc01b035569,51.215329,3.249471,SM-A600FN,False,1548251027,2019-01-23 14:45:21.055,1548251121055,_,_


In [39]:
leegKr = pd.read_excel("D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/leegvglKortrijk.xlsx")

In [40]:
leegBr = pd.read_excel("D:/Users/beheerder/Documents/Floor/UGent/2018-2019/stage/experiment verloop/leegvglBrugge.xlsx")

## voorbereidingen 

#### omvormen van datetime objecten 

In [41]:
df['startTime'] = pd.to_datetime(df['startTime'])
df['endTime'] = pd.to_datetime(df['endTime'])

In [123]:
dfBr['startTime'] = pd.to_datetime(dfBr['startTime'])
dfBr['endTime'] = pd.to_datetime(dfBr['endTime'])

In [43]:
#datum variabele maken zonder tijdstip 

In [44]:
df['dag'] = df.startTime.dt.date

In [124]:
dfBr['dag'] = dfBr.startTime.dt.date

op basis van deze variabele kunnen we dan gaan bepalen: <br>
<ul>(1) week of  weekend </ul>
<ul> (2) schoolweek 1 (normaal beleid) of schoolweek 2 (aangepast beleid)</ul>

In [46]:
#uur variabele aanmaken, idem voor minuut variabele, goed om lestijd - speeltijd - na/school
#deze variabele wijst daarbij op het begin van de app-event. 

In [125]:
dfBr['uur'] = dfBr.startTime.dt.hour
dfBr['minuut'] = dfBr.startTime.dt.minute 

In [48]:
df['uur'] = df.startTime.dt.hour
df['minuut'] = df.startTime.dt.minute 

In [49]:
#duur variabele aanmaken: eindtijd - starttijd / 1000 => in seconden. 

In [50]:
op basis hiervan kun je de smartphoneduur berekenen (apptime) voor de verschillende momenten. 

SyntaxError: invalid syntax (<ipython-input-50-943354ea29e5>, line 1)

In [51]:
df['duur'] = (df.endTimeMillis - df.startTimeMillis)/1000

In [126]:
dfBr['duur'] = (dfBr.endTimeMillis - dfBr.startTimeMillis)/1000

In [53]:
df.head()

,application,battery,data_version,endTime,endTimeMillis,id,latitude,longitude,model,notification,session,startTime,startTimeMillis,studyKey,surveyId,uur,minuut,dag,duur
0,com.snapchat.android,67,1.5,2019-01-17 16:32:16.801,1.547739e+12,a29814bf-bd90-48e6-b42e-368285fd7579,0.0,0.0,MYA-L41,False,1547738718,2019-01-17 16:31:56.753,1.547739e+12,_,_,16,31,2019-01-17,20.048
1,com.snapchat.android,61,1.5,2019-01-17 17:01:53.643,1.547741e+12,a29814bf-bd90-48e6-b42e-368285fd7579,0.0,0.0,MYA-L41,False,1547740566,2019-01-17 17:00:34.860,1.547741e+12,_,_,17,0,2019-01-17,78.783
2,com.twitter.android,61,1.5,2019-01-17 17:15:23.889,1.547742e+12,a29814bf-bd90-48e6-b42e-368285fd7579,0.0,0.0,MYA-L41,False,1547740566,2019-01-17 17:05:48.295,1.547741e+12,_,_,17,5,2019-01-17,575.594
3,com.snapchat.android,46,1.5,2019-01-17 17:56:24.387,1.547744e+12,a29814bf-bd90-48e6-b42e-368285fd7579,0.0,0.0,MYA-L41,False,1547741927,2019-01-17 17:56:20.227,1.547744e+12,_,_,17,56,2019-01-17,4.160
4,com.android.mms,42,1.5,2019-01-17 18:32:38.196,1.547746e+12,a29814bf-bd90-48e6-b42e-368285fd7579,0.0,0.0,MYA-L41,False,1547745578,2019-01-17 18:32:11.967,1.547746e+12,_,_,18,32,2019-01-17,26.229


In [127]:
dfBr.head()

,application,battery,data_version,endTime,endTimeMillis,id,latitude,longitude,model,notification,session,startTime,startTimeMillis,studyKey,surveyId,dag,uur,minuut,duur
0,com.android.systemui,29.0,1.5,2019-01-23 19:07:26.649,1548266846649,00805caf-ff20-4bfb-964f-8bc01b035569,51.198560,3.240879,SM-A600FN,False,1548266758,2019-01-23 19:07:24.222,1548266844222,_,_,2019-01-23,19,7,2.427
1,com.android.chrome,26.0,1.5,2019-01-23 19:14:53.040,1548267293040,00805caf-ff20-4bfb-964f-8bc01b035569,51.198560,3.240879,SM-A600FN,False,1548267175,2019-01-23 19:14:37.246,1548267277246,_,_,2019-01-23,19,14,15.794
2,com.facebook.orca,89.0,1.5,2019-01-23 14:44:38.098,1548251078098,00805caf-ff20-4bfb-964f-8bc01b035569,51.215329,3.249471,SM-A600FN,False,1548251027,2019-01-23 14:43:57.623,1548251037623,_,_,2019-01-23,14,43,40.475
3,com.sec.android.app.sbrowser,89.0,1.5,2019-01-23 14:45:20.986,1548251120986,00805caf-ff20-4bfb-964f-8bc01b035569,51.215329,3.249471,SM-A600FN,False,1548251027,2019-01-23 14:45:20.968,1548251120968,_,_,2019-01-23,14,45,0.018
4,com.sec.android.app.sbrowser,89.0,1.5,2019-01-23 14:45:22.545,1548251122545,00805caf-ff20-4bfb-964f-8bc01b035569,51.215329,3.249471,SM-A600FN,False,1548251027,2019-01-23 14:45:21.055,1548251121055,_,_,2019-01-23,14,45,1.490


## Algemene informatie 

In [55]:
#Aantal dagen gelogd, per participant. 

In [56]:
leegKr["log_dag_Kr"] = df.groupby('id')['dag'].nunique('dag')
leegKr["log_dag_Kr"].describe()

count    12.000000
mean     14.166667
std       4.589184
min       2.000000
25%      12.500000
50%      15.000000
75%      17.250000
max      18.000000
Name: log_dag_Kr, dtype: float64

In [128]:
leegBr["log_dag_Br"] = dfBr.groupby('id')['dag'].nunique('dag')
leegBr["log_dag_Br"].describe()

count     5.000000
mean     11.800000
std       5.357238
min       7.000000
25%       8.000000
50%       9.000000
75%      16.000000
max      19.000000
Name: log_dag_Br, dtype: float64

In [129]:
leegBr

,log_dag_Br,perdag_sec,perdag_min
id,,,
00805caf-ff20-4bfb-964f-8bc01b035569,8,3772.519000,62.875317
80ee7dd1-f7e8-4340-979f-fa0fc70ca198,7,5657.847167,94.297453
b261046b-b4dd-41b4-ae65-fa18e9deca9c,19,15697.074556,261.617909
b346e6ab-1f4c-4c96-9780-af6d6fa1d1f9,16,10193.952556,169.899209
b59216b0-c1a0-4f56-8eb7-17d299c14948,9,4286.300000,71.438333


In [58]:
# Gemiddelde tijd (sec) over dagen heen, per participant
leegKr["perdag_sec"] = df.groupby(['id','dag'])['duur'].sum().groupby('id').mean()
#gemiddelde tijd (minutes) over dagen heen, per participant 
leegKr["perdag_min"] = leegKr["perdag_sec"]/60

In [130]:
# Gemiddelde tijd (sec) over dagen heen, per participant
leegBr["perdag_sec"] = dfBr.groupby(['id','dag'])['duur'].sum().groupby('id').mean()
#gemiddelde tijd (minutes) over dagen heen, per participant 
leegBr["perdag_min"] = leegBr["perdag_sec"]/60

In [59]:
leegKr.head()

,log_dag_Kr,perdag_sec,perdag_min
id,,,
09c20aee-6789-40f2-ab64-95bcd190663c,18,10847.300389,180.788340
0def5422-9828-43d4-8aa9-fd949779ba71,15,10894.386800,181.573113
12d6ca9f-b961-48cf-a41a-0f5824c2b444,17,5725.379059,95.422984
1e6526cd-4609-4c7f-9cea-4f4c0b47c24a,15,13572.339400,226.205657
225d33b4-b261-4b7d-a18c-ed1909680462,11,8995.026636,149.917111


In [131]:
leegBr

,log_dag_Br,perdag_sec,perdag_min
id,,,
00805caf-ff20-4bfb-964f-8bc01b035569,8,4550.315875,75.838598
80ee7dd1-f7e8-4340-979f-fa0fc70ca198,7,6338.109000,105.635150
b261046b-b4dd-41b4-ae65-fa18e9deca9c,19,19462.185105,324.369752
b346e6ab-1f4c-4c96-9780-af6d6fa1d1f9,16,13976.602812,232.943380
b59216b0-c1a0-4f56-8eb7-17d299c14948,9,10414.753111,173.579219


In [543]:
leegKr["perdag_min"].describe()

count     12.000000
mean     175.866376
std       72.881602
min       76.138167
25%      117.638179
50%      169.928237
75%      233.807378
max      296.715681
Name: perdag_min, dtype: float64

In [544]:
leeg.head()

,log_dag,perdag_sec,perdag_min
id,,,
09c20aee-6789-40f2-ab64-95bcd190663c,18,10847.300389,180.788340
0def5422-9828-43d4-8aa9-fd949779ba71,15,10894.386800,181.573113
12d6ca9f-b961-48cf-a41a-0f5824c2b444,17,5725.379059,95.422984
1e6526cd-4609-4c7f-9cea-4f4c0b47c24a,15,13572.339400,226.205657
225d33b4-b261-4b7d-a18c-ed1909680462,11,8995.026636,149.917111


In [545]:
leeg.columns = ["log_dag", "perdag_sec_over3wkn", "perdag_min_over3wkn"]

In [ ]:
#eerst nog eens gemiddeld gebruik per dag bekijken om te zien of er al eerdere indicaties zijn. 

In [546]:
df.groupby(['id','dag'])['duur'].sum().groupby('dag').mean()/60

dag
2019-01-14    117.970969
2019-01-15    177.644564
2019-01-16    192.166420
2019-01-17    154.294832
2019-01-18    160.234053
2019-01-19    202.427562
2019-01-20    231.779997
2019-01-21    151.301167
2019-01-22    176.824485
2019-01-23    258.788267
2019-01-24    182.217765
2019-01-25    277.459070
2019-01-26    216.640069
2019-01-27    246.139355
2019-01-28    103.963947
2019-01-29    166.504125
2019-01-30    220.328822
2019-01-31     61.591258
Name: duur, dtype: float64

In [ ]:
14 is het moment waarop het geinstalleerd is, dus waarschijnlijk niet volledig. 
31 is het moment waarop de data genomen is, dus ook niet volledig. 
Deze dagen eruit laten? 
in het algemeen redelijk gelijkaardig, maar wel opvallend 25e was lesvrije dag, 
daar heb je een piek in meeste gebruik. 

In het algemeen dus niet echt een effect te zien van smartphone beleid volgens mij, 
maar misschien zal dit er wel zijn als de opsplitsing wordt gemaakt. 

In [132]:
dfBr["dag"].value_counts()

2019-01-30    1356
2019-01-31    1091
2019-01-28     967
2019-02-01     950
2019-02-06     928
2019-02-02     878
2019-02-03     848
2019-02-04     816
2019-02-05     700
2019-01-29     652
2019-01-24     639
2019-01-27     618
2019-01-23     598
2019-02-07     512
2019-01-25     460
2019-02-08     407
2019-01-26     369
2019-02-09     260
2019-02-10     103
Name: dag, dtype: int64

## Data onderverdelen aan de hand van week tov weekend (= moment )

In [76]:
#welke dagen zitten erin => welke dagen waren weekdagen en welke dagen waren weekenddagen? 
#Kortrijk = allemaal dagen in januari, dus puur focusen op dag. 
df['day'] = df.startTime.dt.day
df['naam_dag'] = df.startTime.dt.weekday_name
df['day'].unique()

array([17, 16, 18, 14, 15, 19, 21, 20, 22, 23, 24, 25, 26, 27, 28, 30, 29,
       31], dtype=int64)

In [133]:
dfBr['day'] = dfBr.startTime.dt.day
dfBr['naam_dag'] = dfBr.startTime.dt.weekday_name
dfBr['day'].unique()

array([23, 24, 25, 26, 27, 28, 30, 29, 31,  1,  2,  6,  5,  3,  4,  7,  9,
       10,  8], dtype=int64)

23 = woensdag, 24 = donderdag, 25 = vrijdag (week 1)
26 = zaterdag, 27 = zondag (weekend 1)
28 = maandag, 29 = dinsdag, 30 = woensdag (week 1)
31 = donderdag (week 2)
1 = vrijdag (week 2)
2 = zaterdag (week 2)
3 = zondag (week 2)
4 = maandag (week 2)
5 = dinsdag (week 2)
6 = woensdag (week 2) 
7 = donderdag (week 3)
8 = vrijdag (week 3) 
9 = zaterdag (week 3)
10 = zondag (week 3)

In [ ]:
#eerst en vooral de eerste en de laatste logdag eruithalen, omdat deze onvolledig zijn. 

In [77]:
df_adjust = df[df['day'] != 14]

In [78]:
df_adjust = df_adjust[df_adjust['day'] != 31]

In [79]:
df_adjust['day'].unique()

array([17, 16, 18, 15, 19, 21, 20, 22, 23, 24, 25, 26, 27, 28, 30, 29],
      dtype=int64)

In [80]:
#wat is week & wat is weekend? 14 januari tot 31 januari 
# weekend1 = 19 - 20
# weekend2 = 26-27 (+ 25 vrije dag) !!dus vrije dag is ook gezien als vrije dag! 

In [81]:
df_adjust['moment'] = np.where((df_adjust.day == 19)|(df_adjust.day == 20)|(df_adjust.day == 25)|(df_adjust.day == 26)|(df_adjust.day == 27), 'WEEKEND', 'WEEK')

In [135]:
dfBr['moment'] = np.where((dfBr.day == 26)|(dfBr.day == 27)|(dfBr.day == 2)|(dfBr.day == 3)|(dfBr.day == 9)|(dfBr.day == 10), 'WEEKEND', 'WEEK')

#### gemiddelde voor week & weekend per participant (long format)

In [82]:
mm = df_adjust.groupby(['id','moment','day'])['duur'].sum()/60
urengebruik = mm.groupby(['id', 'moment']).mean()
urengebruik = pd.DataFrame(data=urengebruik)

In [70]:
mm = dfBr.groupby(['id','moment','day'])['duur'].sum()/60
urengebruik = mm.groupby(['id', 'moment']).mean()
urengebruik = pd.DataFrame(data=urengebruik)

In [107]:
mm = dfBr.groupby(['id','dag','moment'])['duur'].sum()/60
urengebruik = mm.groupby(['moment']).mean()
urengebruik = pd.DataFrame(data=urengebruik)

In [108]:
urengebruik

,duur
moment,
WEEK,149.831670
WEEKEND,149.980412


In [553]:
urengebruik = urengebruik.reset_index()

In [554]:
#to wide format 
urengebruik_wide = urengebruik.pivot(index="id", columns="moment", values="duur")

In [556]:
urengebruik_wide.columns = ["min_WEEK_3wkn", "min_WEEKEND_3wkn"]

In [557]:
leeg = pd.merge(leeg, urengebruik_wide, on = "id")

In [394]:
df_adjust.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32640 entries, 0 to 34572
Data columns (total 23 columns):
application        32640 non-null object
battery            32640 non-null int64
data_version       32640 non-null float64
endTime            32640 non-null datetime64[ns]
endTimeMillis      32640 non-null float64
id                 32640 non-null object
latitude           32640 non-null float64
longitude          32640 non-null float64
model              32640 non-null object
notification       32640 non-null bool
session            32640 non-null int64
startTime          32640 non-null datetime64[ns]
startTimeMillis    32640 non-null float64
studyKey           32640 non-null object
surveyId           32640 non-null object
dag                32640 non-null object
uur                32640 non-null int64
minuut             32640 non-null int64
duur               32640 non-null float64
day                32640 non-null int64
naam_dag           32640 non-null object
moment         

In [100]:
dfBr["id"].value_counts()

b261046b-b4dd-41b4-ae65-fa18e9deca9c    1446
b346e6ab-1f4c-4c96-9780-af6d6fa1d1f9    1063
80ee7dd1-f7e8-4340-979f-fa0fc70ca198    1037
00805caf-ff20-4bfb-964f-8bc01b035569     866
b59216b0-c1a0-4f56-8eb7-17d299c14948     245
Name: id, dtype: int64

In [101]:
dfBr["duur"].mean()

65.6472147305131

In [102]:
dfBr

,application,battery,data_version,endTime,endTimeMillis,id,latitude,longitude,model,notification,...,startTimeMillis,studyKey,surveyId,dag,uur,minuut,duur,day,naam_dag,moment
0,com.google.android.gm,95.0,1.5,2019-01-23 08:51:46.738,1548229906738,00805caf-ff20-4bfb-964f-8bc01b035569,51.215638,3.248788,SM-A600FN,False,...,1548229855060,_,_,2019-01-23,8,50,51.678,23,Wednesday,WEEK
1,be.ugent.mobiledna,95.0,1.5,2019-01-23 08:52:06.760,1548229926760,00805caf-ff20-4bfb-964f-8bc01b035569,51.215638,3.248788,SM-A600FN,False,...,1548229907031,_,_,2019-01-23,8,51,19.729,23,Wednesday,WEEK
2,be.ugent.mobiledna,95.0,1.5,2019-01-23 09:12:58.410,1548231178410,00805caf-ff20-4bfb-964f-8bc01b035569,51.215593,3.248963,SM-A600FN,False,...,1548231174569,_,_,2019-01-23,9,12,3.841,23,Wednesday,WEEK
3,be.smartschool.mobile,95.0,1.5,2019-01-23 09:14:27.200,1548231267200,00805caf-ff20-4bfb-964f-8bc01b035569,51.215593,3.248963,SM-A600FN,False,...,1548231220760,_,_,2019-01-23,9,13,46.440,23,Wednesday,WEEK
4,be.ugent.mobiledna,85.0,1.5,2019-01-23 15:02:16.939,1548252136939,00805caf-ff20-4bfb-964f-8bc01b035569,51.207878,3.248880,SM-A600FN,False,...,1548252136020,_,_,2019-01-23,15,2,0.919,23,Wednesday,WEEK
5,com.android.systemui,85.0,1.5,2019-01-23 15:02:17.546,1548252137546,00805caf-ff20-4bfb-964f-8bc01b035569,51.207878,3.248880,SM-A600FN,False,...,1548252136950,_,_,2019-01-23,15,2,0.596,23,Wednesday,WEEK
6,com.android.systemui,82.0,1.5,2019-01-23 15:27:15.053,1548253635053,00805caf-ff20-4bfb-964f-8bc01b035569,51.207878,3.248880,SM-A600FN,False,...,1548253633311,_,_,2019-01-23,15,27,1.742,23,Wednesday,WEEK
7,com.android.systemui,82.0,1.5,2019-01-23 15:27:18.024,1548253638024,00805caf-ff20-4bfb-964f-8bc01b035569,51.207878,3.248880,SM-A600FN,False,...,1548253636698,_,_,2019-01-23,15,27,1.326,23,Wednesday,WEEK
8,com.twitter.android,89.0,1.5,2019-01-23 14:45:51.651,1548251151651,00805caf-ff20-4bfb-964f-8bc01b035569,51.215329,3.249471,SM-A600FN,False,...,1548251124200,_,_,2019-01-23,14,45,27.451,23,Wednesday,WEEK
9,com.android.systemui,86.0,1.5,2019-01-23 14:45:52.080,1548251152080,00805caf-ff20-4bfb-964f-8bc01b035569,51.215329,3.249471,SM-A600FN,False,...,1548251151666,_,_,2019-01-23,14,45,0.414,23,Wednesday,WEEK


## Data onderverdelen aan de hand van lestijd, speeltijd, voor-na-school (=dagverloop)

In [ ]:
voor 8u20 = voor schooltijd 
8u20 - 9u10 = lesuur 1
9u10 - 10u05 = lesuur 2
10u05 - 10u15 = speeltijd 
10u15 - 11u05 = lesuur 3
11u05 - 11u55 = lesuur 4
11u55 - 12u50 = middagpauze
12u50 - 13u40 = lesuur 5
13u40 - 14u40 = lesuur 6
14u40 - 14u50 = speeltijd
14u50 - 15u40 = lesuur 7
15u40 - 16u30 = lesuur 8
> 17u = na schooltijd 

In [83]:
def dagverloop (row):
    if ((row["moment"] == "WEEK") & ((row["naam_dag"] ==  "Monday")|(row["naam_dag"] ==  "Tuesday")|(row["naam_dag"] ==  "Thursday")|(row["naam_dag"] ==  "Friday"))):
        if (row["uur"] < 8):
            return "vóór"
        if ((row["uur"] == 8) & (row["minuut"] < 20)):
            return "vóór"
        if ((row["uur"] == 8) & (row["minuut"] == 20)):
            return "lesuur 1"
        if ((row["uur"] == 8) & (row["minuut"] > 20)):
            return "lesuur 1"
        if ((row["uur"] == 9) & (row["minuut"] < 10)):
            return "lesuur 1"
        if ((row["uur"] == 9) & (row["minuut"] == 10)):
            return "lesuur 2"
        if ((row["uur"] == 9) & (row["minuut"] > 10)):
            return "lesuur 2"
        if ((row["uur"] == 10) & (row["minuut"] < 5)):
            return "lesuur 2"
        if ((row["uur"] == 10) & (row["minuut"] == 5)):
            return "speeltijd 1"
        if (((row["uur"] == 10) & (row["minuut"] > 5)) & ((row["uur"] == 10) & (row["minuut"] < 15))):
            return "speeltijd 1"
        if ((row["uur"] == 10) & (row["minuut"] > 15)):
            return "lesuur 3"
        if ((row["uur"] == 11) & (row["minuut"] < 5)):
            return "lesuur 3"
        if ((row["uur"] == 11) & (row["minuut"] == 5)):
            return "lesuur 4"
        if ((row["uur"] == 11) & (row["minuut"] > 5)):
            return "lesuur 4"
        if ((row["uur"] == 11) & (row["minuut"] < 55)):
            return "lesuur 4"
        if ((row["uur"] == 11) & (row["minuut"] == 55)):
            return "middagpauze"
        if ((row["uur"] == 11) & (row["minuut"] > 55)):
            return "middagpauze"    
        if ((row["uur"] == 12) & (row["minuut"] < 50)):
            return "middagpauze"
        if ((row["uur"] == 12) & (row["minuut"] == 50)):
            return "lesuur 5" 
        if ((row["uur"] == 12) & (row["minuut"] > 50)):
            return "lesuur 5"
        if ((row["uur"] == 13) & (row["minuut"] < 40)):
            return "lesuur 5"
        if ((row["uur"] == 13) & (row["minuut"] == 40)):
            return "lesuur 6"
        if ((row["uur"] == 13) & (row["minuut"] > 40)):
            return "lesuur 6" 
        if ((row["uur"] == 14) & (row["minuut"] < 40)):
            return "lesuur 6"
        if ((row["uur"] == 14) & (row["minuut"] == 40)):
            return "speeltijd 2"
        if (((row["uur"] == 14) & (row["minuut"] > 40)) & ((row["uur"] <= 14) & (row["minuut"] < 50))):
            return "speeltijd 2" 
        if ((row["uur"] == 14) & (row["minuut"] > 50)):
            return "lesuur 7"
        if ((row["uur"] == 15) & (row["minuut"] < 40)):
            return "lesuur 7"
        if ((row["uur"] == 15) & (row["minuut"] == 40)):
            return "lesuur 7"
        if ((row["uur"] == 15) & (row["minuut"] > 40)):
            return "lesuur 8"
        if ((row["uur"] == 16) & (row["minuut"] < 30)):
            return "lesuur 8"
        if ((row["uur"] == 16) & (row["minuut"] == 30)):
            return "lesuur 8"
        if ((row["uur"] == 16 ) & (row["minuut"] > 30)):
            return "na"
        if (row["uur"] >= 17):
            return "na"
        
    if ((row["moment"] == "WEEK") & (row["naam_dag"] ==  "Wednesday")):
        if (row["uur"] < 8):
            return "vóór"
        if ((row["uur"] == 8) & (row["minuut"] < 20)):
            return "vóór"
        if ((row["uur"] == 8) & (row["minuut"] == 20)):
            return "lesuur 1"
        if ((row["uur"] == 8) & (row["minuut"] > 20)):
            return "lesuur 1"
        if ((row["uur"] == 9) & (row["minuut"] < 10)):
            return "lesuur 1"
        if ((row["uur"] == 9) & (row["minuut"] == 10)):
            return "lesuur 2"
        if ((row["uur"] == 9) & (row["minuut"] > 10)):
            return "lesuur 2"
        if ((row["uur"] == 10) & (row["minuut"] < 5)):
            return "lesuur 2"
        if ((row["uur"] == 10) & (row["minuut"] == 5)):
            return "speeltijd 1"
        if (((row["uur"] == 10) & (row["minuut"] > 5)) & ((row["uur"] == 10) & (row["minuut"] < 15))):
            return "speeltijd 1"
        if ((row["uur"] == 10) & (row["minuut"] > 15)):
            return "lesuur 3"
        if ((row["uur"] == 11) & (row["minuut"] < 5)):
            return "lesuur 3"
        if ((row["uur"] == 11) & (row["minuut"] == 5)):
            return "lesuur 4"
        if ((row["uur"] == 11) & (row["minuut"] > 5)):
            return "lesuur 4"
        if ((row["uur"] == 11) & (row["minuut"] < 55)):
            return "lesuur 4"
        if ((row["uur"] == 11) & (row["minuut"] == 55)):
            return "na"
        if ((row["uur"] == 11) & (row["minuut"] > 55)):
            return "na"
        if (row["uur"] >= 12):
            return "na"
        
    if (row["moment"] == "WEEKEND"):
        return "weekend"
        
        

In [84]:
df_adjust["dagverloop"] = df_adjust.apply(dagverloop, axis=1)

In [136]:
dfBr["dagverloop"] = dfBr.apply(dagverloop, axis = 1)

In [567]:
def dagverloop_algemeen (row):
    if ((row["moment"] == "WEEK") & ((row["naam_dag"] ==  "Monday")|(row["naam_dag"] ==  "Tuesday")|(row["naam_dag"] ==  "Thursday")|(row["naam_dag"] ==  "Friday"))):
        if (row["uur"] < 8):
            return "vóór/na"
        if ((row["uur"] == 8) & (row["minuut"] < 20)):
            return "vóór/na"
        if ((row["uur"] == 8) & (row["minuut"] == 20)):
            return "les"
        if ((row["uur"] == 8) & (row["minuut"] > 20)):
            return "les"
        if ((row["uur"] == 9) & (row["minuut"] < 10)):
            return "les"
        if ((row["uur"] == 9) & (row["minuut"] == 10)):
            return "les"
        if ((row["uur"] == 9) & (row["minuut"] > 10)):
            return "les"
        if ((row["uur"] == 10) & (row["minuut"] < 5)):
            return "les"
        if ((row["uur"] == 10) & (row["minuut"] == 5)):
            return "pauze"
        if (((row["uur"] == 10) & (row["minuut"] > 5)) & ((row["uur"] == 10) & (row["minuut"] < 15))):
            return "pauze"
        if ((row["uur"] == 10) & (row["minuut"] > 15)):
            return "les"
        if ((row["uur"] == 11) & (row["minuut"] < 5)):
            return "les"
        if ((row["uur"] == 11) & (row["minuut"] == 5)):
            return "les"
        if ((row["uur"] == 11) & (row["minuut"] > 5)):
            return "les"
        if ((row["uur"] == 11) & (row["minuut"] < 55)):
            return "les"
        if ((row["uur"] == 11) & (row["minuut"] == 55)):
            return "pauze"
        if ((row["uur"] == 11) & (row["minuut"] > 55)):
            return "pauze"    
        if ((row["uur"] == 12) & (row["minuut"] < 50)):
            return "pauze"
        if ((row["uur"] == 12) & (row["minuut"] == 50)):
            return "les" 
        if ((row["uur"] == 12) & (row["minuut"] > 50)):
            return "les"
        if ((row["uur"] == 13) & (row["minuut"] < 40)):
            return "les"
        if ((row["uur"] == 13) & (row["minuut"] == 40)):
            return "les"
        if ((row["uur"] == 13) & (row["minuut"] > 40)):
            return "les" 
        if ((row["uur"] == 14) & (row["minuut"] < 40)):
            return "les"
        if ((row["uur"] == 14) & (row["minuut"] == 40)):
            return "pauze"
        if (((row["uur"] == 14) & (row["minuut"] > 40)) & ((row["uur"] <= 14) & (row["minuut"] < 50))):
            return "pauze" 
        if ((row["uur"] == 14) & (row["minuut"] > 50)):
            return "les"
        if ((row["uur"] == 15) & (row["minuut"] < 40)):
            return "les"
        if ((row["uur"] == 15) & (row["minuut"] == 40)):
            return "les"
        if ((row["uur"] == 15) & (row["minuut"] > 40)):
            return "les"
        if ((row["uur"] == 16) & (row["minuut"] < 30)):
            return "les"
        if ((row["uur"] == 16) & (row["minuut"] == 30)):
            return "les"
        if ((row["uur"] == 16 ) & (row["minuut"] > 30)):
            return "vóór/na"
        if (row["uur"] >= 17):
            return "vóór/na"
        
    if ((row["moment"] == "WEEK") & (row["naam_dag"] ==  "Wednesday")):
        if (row["uur"] < 8):
            return "vóór/na"
        if ((row["uur"] == 8) & (row["minuut"] < 20)):
            return "vóór/na"
        if ((row["uur"] == 8) & (row["minuut"] == 20)):
            return "les"
        if ((row["uur"] == 8) & (row["minuut"] > 20)):
            return "les"
        if ((row["uur"] == 9) & (row["minuut"] < 10)):
            return "les"
        if ((row["uur"] == 9) & (row["minuut"] == 10)):
            return "les"
        if ((row["uur"] == 9) & (row["minuut"] > 10)):
            return "les"
        if ((row["uur"] == 10) & (row["minuut"] < 5)):
            return "les"
        if ((row["uur"] == 10) & (row["minuut"] == 5)):
            return "pauze"
        if (((row["uur"] == 10) & (row["minuut"] > 5)) & ((row["uur"] == 10) & (row["minuut"] < 15))):
            return "pauze"
        if ((row["uur"] == 10) & (row["minuut"] > 15)):
            return "les"
        if ((row["uur"] == 11) & (row["minuut"] < 5)):
            return "les"
        if ((row["uur"] == 11) & (row["minuut"] == 5)):
            return "les"
        if ((row["uur"] == 11) & (row["minuut"] > 5)):
            return "les"
        if ((row["uur"] == 11) & (row["minuut"] < 55)):
            return "les"
        if ((row["uur"] == 11) & (row["minuut"] == 55)):
            return "vóór/na"
        if ((row["uur"] == 11 ) & (row["minuut"] > 55)):
            return "vóór/na"
        if (row["uur"] == 12):
            return "vóór/na"
        if (row["uur"] > 12):
            return "vóór/na"
        
    if (row["moment"] == "WEEKEND"):
        return "weekend"

In [568]:
df_adjust["dagverloop_algemeen"] = df_adjust.apply(dagverloop_algemeen, axis=1)

In [137]:
dfBr

,application,battery,data_version,endTime,endTimeMillis,id,latitude,longitude,model,notification,...,studyKey,surveyId,dag,uur,minuut,duur,day,naam_dag,moment,dagverloop
0,com.android.systemui,29.0,1.5,2019-01-23 19:07:26.649,1548266846649,00805caf-ff20-4bfb-964f-8bc01b035569,51.198560,3.240879,SM-A600FN,False,...,_,_,2019-01-23,19,7,2.427,23,Wednesday,WEEK,na
1,com.android.chrome,26.0,1.5,2019-01-23 19:14:53.040,1548267293040,00805caf-ff20-4bfb-964f-8bc01b035569,51.198560,3.240879,SM-A600FN,False,...,_,_,2019-01-23,19,14,15.794,23,Wednesday,WEEK,na
2,com.facebook.orca,89.0,1.5,2019-01-23 14:44:38.098,1548251078098,00805caf-ff20-4bfb-964f-8bc01b035569,51.215329,3.249471,SM-A600FN,False,...,_,_,2019-01-23,14,43,40.475,23,Wednesday,WEEK,na
3,com.sec.android.app.sbrowser,89.0,1.5,2019-01-23 14:45:20.986,1548251120986,00805caf-ff20-4bfb-964f-8bc01b035569,51.215329,3.249471,SM-A600FN,False,...,_,_,2019-01-23,14,45,0.018,23,Wednesday,WEEK,na
4,com.sec.android.app.sbrowser,89.0,1.5,2019-01-23 14:45:22.545,1548251122545,00805caf-ff20-4bfb-964f-8bc01b035569,51.215329,3.249471,SM-A600FN,False,...,_,_,2019-01-23,14,45,1.490,23,Wednesday,WEEK,na
5,com.facebook.katana,86.0,1.5,2019-01-23 14:47:21.135,1548251241135,00805caf-ff20-4bfb-964f-8bc01b035569,51.215593,3.248963,SM-A600FN,False,...,_,_,2019-01-23,14,46,70.133,23,Wednesday,WEEK,na
6,com.facebook.katana,86.0,1.5,2019-01-23 14:47:29.613,1548251249613,00805caf-ff20-4bfb-964f-8bc01b035569,51.215593,3.248963,SM-A600FN,False,...,_,_,2019-01-23,14,47,5.511,23,Wednesday,WEEK,na
7,com.android.chrome,86.0,1.5,2019-01-23 14:48:06.998,1548251286998,00805caf-ff20-4bfb-964f-8bc01b035569,51.215593,3.248963,SM-A600FN,False,...,_,_,2019-01-23,14,47,13.951,23,Wednesday,WEEK,na
8,com.android.systemui,86.0,1.5,2019-01-23 14:48:13.544,1548251293544,00805caf-ff20-4bfb-964f-8bc01b035569,51.215593,3.248963,SM-A600FN,False,...,_,_,2019-01-23,14,48,0.421,23,Wednesday,WEEK,na
9,com.android.systemui,86.0,1.5,2019-01-23 14:49:28.543,1548251368543,00805caf-ff20-4bfb-964f-8bc01b035569,51.215593,3.248963,SM-A600FN,False,...,_,_,2019-01-23,14,49,1.369,23,Wednesday,WEEK,na


In [572]:
df_adjust.to_excel("dataKortrijk5022019.xlsx")

In [138]:
dfBr.to_excel("dataBrugge26022019.xlsx")

Dit is nu een dataframe waarmee kan worden mee begonnen. 

## Data onderverdelen aan de hand van schoolweek 1 tov schoolweek 2 (= beleid)

In [85]:
#lesweek 1 = 14 tot 20 (gaat over de volledige week: dus inclusief weekend, kan onderscheiden worden adhv combo met WEEK & WEEKEND)
#lesweek 2 = 21 tot 27
#lesweek 3 = 28 tot 30

In [86]:
functie maken op basis van 'day'

SyntaxError: invalid syntax (<ipython-input-86-5b51854b9563>, line 1)

In [87]:
def lesweek(row):
    if row["day"] == 14:
        return "lesweek 1"
    if row["day"] == 15:
        return "lesweek 1"
    if row["day"] == 16:
        return "lesweek 1"
    if row["day"] == 17:
        return "lesweek 1"
    if row["day"] == 18:
        return "lesweek 1"
    if row["day"] == 19:
        return "lesweek 1"
    if row["day"] == 20:
        return "lesweek 1"
    if row["day"] == 21:
        return "lesweek 2"
    if row["day"] == 22:
        return "lesweek 2"
    if row["day"] == 23:
        return "lesweek 2"
    if row["day"] == 24:
        return "lesweek 2"
    if row["day"] == 25:
        return "lesweek 2"
    if row["day"] == 26:
        return "lesweek 2"
    if row["day"] == 27:
        return "lesweek 2"
    if row["day"] == 28:
        return "lesweek 3"
    if row["day"] == 29:
        return "lesweek 3"
    if row["day"] == 30:
        return "lesweek 3"

In [139]:
def lesweek(row):
    if row["day"] == 23:
        return "lesweek 1"
    if row["day"] == 24:
        return "lesweek 1"
    if row["day"] == 25:
        return "lesweek 1"
    if row["day"] == 26:
        return "lesweek 1"
    if row["day"] == 27:
        return "lesweek 1"
    if row["day"] == 28:
        return "lesweek 1"
    if row["day"] == 29:
        return "lesweek 1"
    if row["day"] == 30:
        return "lesweek 1"
    if row["day"] == 31:
        return "lesweek 2"
    if row["day"] == 1:
        return "lesweek 2"
    if row["day"] == 2:
        return "lesweek 2"
    if row["day"] == 3:
        return "lesweek 2"
    if row["day"] == 4:
        return "lesweek 2"
    if row["day"] == 5:
        return "lesweek 2"
    if row["day"] == 6:
        return "lesweek 2"
    if row["day"] == 7:
        return "lesweek 3"
    if row["day"] == 8:
        return "lesweek 3"
    if row["day"] == 9:
        return "lesweek 3"
    if row["day"] == 10:
        return "lesweek 3"

In [88]:
df_adjust["beleid"] = df_adjust.apply(lesweek, axis=1)

In [140]:
dfBr["beleid"] = dfBr.apply(lesweek, axis = 1)

In [143]:
mm = dfBr.groupby(['id','dagverloop','beleid','moment','day'])['duur'].sum()/60
urengebruik = mm.groupby(['id', 'dagverloop','moment', 'beleid']).mean()
urengebruik = pd.DataFrame(data=urengebruik)

In [144]:
urengebruik = urengebruik.reset_index()

In [145]:
urengebruik.head()

,id,dagverloop,moment,beleid,duur
0,00805caf-ff20-4bfb-964f-8bc01b035569,lesuur 1,WEEK,lesweek 1,1.295208
1,00805caf-ff20-4bfb-964f-8bc01b035569,lesuur 2,WEEK,lesweek 1,0.739250
2,00805caf-ff20-4bfb-964f-8bc01b035569,lesuur 5,WEEK,lesweek 1,11.035822
3,00805caf-ff20-4bfb-964f-8bc01b035569,lesuur 6,WEEK,lesweek 1,1.145950
4,00805caf-ff20-4bfb-964f-8bc01b035569,lesuur 7,WEEK,lesweek 1,8.791217


In [146]:
specifiek = urengebruik[urengebruik["dagverloop"] == "na"]

In [147]:
specifiek = specifiek[specifiek["beleid"] != "lesweek 3"]

In [148]:
specifiek = specifiek[["id", "dagverloop", "beleid", "duur"]]

In [149]:
specifiek_wide = pd.pivot_table(specifiek, index= ["id"], columns= ["dagverloop","beleid"], values=["duur"])

In [150]:
specifiek_wide

duur            
dagverloop                                    na            
beleid                                 lesweek 1   lesweek 2
id                                                          
00805caf-ff20-4bfb-964f-8bc01b035569   74.846510         NaN
80ee7dd1-f7e8-4340-979f-fa0fc70ca198  110.359029         NaN
b261046b-b4dd-41b4-ae65-fa18e9deca9c  248.702711  196.262077
b346e6ab-1f4c-4c96-9780-af6d6fa1d1f9  129.092361  147.537547
b59216b0-c1a0-4f56-8eb7-17d299c14948  136.619333  123.082733

In [99]:
specifiek_wide.to_excel("specifiek_wide_Kortrijk.xlsx")

In [582]:
urengebruik_wide.columns = [("gem_min_dag", "WEEK1", "lesweek1"),("gem_min_dag", "WEEK1", "lesweek2"),("gem_min_dag", "WEEK3","lesweek3"),("gem_min_dag","WEEKEND1", "lesweek1"),("gem_min_dag","WEEKEND2", "lesweek2")]

In [584]:
leeg = pd.merge(leeg, urengebruik_wide, on = "id")

In [585]:
leeg.head()

,log_dag,perdag_sec_over3wkn,perdag_min_over3wkn,min_WEEK_3wkn,min_WEEKEND_3wkn,"(gem_min_dag, WEEK1, lesweek1)","(gem_min_dag, WEEK1, lesweek2)","(gem_min_dag, WEEK3, lesweek3)","(gem_min_dag, WEEKEND1, lesweek1)","(gem_min_dag, WEEKEND2, lesweek2)"
id,,,,,,,,,,
09c20aee-6789-40f2-ab64-95bcd190663c,18,10847.300389,180.788340,177.365427,176.416037,135.226908,203.603979,198.565383,157.522617,189.011650
0def5422-9828-43d4-8aa9-fd949779ba71,15,10894.386800,181.573113,163.412548,242.034040,154.804592,179.791588,132.328217,271.848392,222.157806
12d6ca9f-b961-48cf-a41a-0f5824c2b444,17,5725.379059,95.422984,100.673555,86.283923,63.371875,156.871433,75.478622,72.465717,95.496061
1e6526cd-4609-4c7f-9cea-4f4c0b47c24a,15,13572.339400,226.205657,188.195076,317.417560,193.540092,214.348112,62.202867,268.898275,349.763750
225d33b4-b261-4b7d-a18c-ed1909680462,11,8995.026636,149.917111,152.741942,157.125717,159.870850,NaN,145.613033,163.626508,144.124133


In [ ]:
Beter op de participant met index 4 eruit te laten, net als de participant met 7.
4 heeft geen infomratie over lesweek 2 (kan dus hiermee geen vergelijking maken)
7 heeft geen informatie over lesweek 2 

In [ ]:
wil deze waarden wel in een excel zetten maar hoe moet je deze benoemen? 

In [390]:
urengebruik_wide.to_excel("beleid.xlsx")

In [391]:
beleid = urengebruik_wide

### week 1:

#### smartphonegebruik

In [616]:
#alles op basis van week 1 
df_week1 = df_adjust[df_adjust["beleid"] == "lesweek 1"]

<ul> algemeen gemiddeld dagelijks </ul>

In [587]:
mm = df_week1.groupby(['id', 'moment','day'])['duur'].sum()/60
algemeen1 = mm.groupby(['id']).mean()
algemeen1 = pd.DataFrame(data=algemeen1)

In [109]:
mm = dfBr.groupby(['id', 'moment','day'])['duur'].sum()/60
algemeen1 = mm.groupby(['id']).mean()
algemeen1 = pd.DataFrame(data=algemeen1)

In [110]:
algemeen1

,duur
id,
00805caf-ff20-4bfb-964f-8bc01b035569,62.875317
80ee7dd1-f7e8-4340-979f-fa0fc70ca198,94.297453
b261046b-b4dd-41b4-ae65-fa18e9deca9c,261.617909
b346e6ab-1f4c-4c96-9780-af6d6fa1d1f9,169.899209
b59216b0-c1a0-4f56-8eb7-17d299c14948,71.438333


In [588]:
algemeen1= algemeen1.reset_index()

In [589]:
algemeen1.columns = ["id", "alg_min_gemiddeld_dagelijks_W1"]

In [592]:
leeg = pd.merge(leeg, algemeen1, on='id')

<ul> algemeen gemiddeld tijdens weekdag vs weekenddag </ul>

In [593]:
mm = df_week1.groupby(['id', 'moment','day'])['duur'].sum()/60
moment1 = mm.groupby(['id', 'moment']).mean()
moment1 = pd.DataFrame(data=moment1)

In [111]:
mm = dfBr.groupby(['id', 'moment','day'])['duur'].sum()/60
moment1 = mm.groupby(['id', 'moment']).mean()
moment1 = pd.DataFrame(data=moment1)

In [112]:
moment1

duur
id                                   moment             
00805caf-ff20-4bfb-964f-8bc01b035569 WEEK      71.175594
                                     WEEKEND   37.974483
80ee7dd1-f7e8-4340-979f-fa0fc70ca198 WEEK      86.639397
                                     WEEKEND  132.587733
b261046b-b4dd-41b4-ae65-fa18e9deca9c WEEK     272.110826
                                     WEEKEND  224.892700
b346e6ab-1f4c-4c96-9780-af6d6fa1d1f9 WEEK     162.507443
                                     WEEKEND  195.770392
b59216b0-c1a0-4f56-8eb7-17d299c14948 WEEK      71.438333

In [113]:
moment1_wide = pd.pivot_table(moment1, index= ["id"], columns= ["moment"], values=["duur"])

In [114]:
moment1_wide

duur            
moment                                      WEEK     WEEKEND
id                                                          
00805caf-ff20-4bfb-964f-8bc01b035569   71.175594   37.974483
80ee7dd1-f7e8-4340-979f-fa0fc70ca198   86.639397  132.587733
b261046b-b4dd-41b4-ae65-fa18e9deca9c  272.110826  224.892700
b346e6ab-1f4c-4c96-9780-af6d6fa1d1f9  162.507443  195.770392
b59216b0-c1a0-4f56-8eb7-17d299c14948   71.438333         NaN

In [597]:
moment1_wide.columns = [("gem.min/dag", "WEEK1"), ("gem.min/dag", "WEEKEND1")]

In [599]:
leeg = pd.merge(leeg, moment1_wide, on='id')

<ul> algemeen gemiddeld tijdens les, tijdens pauze, voor/na school: per dag vd week </ul>

In [600]:
mm = df_week1.groupby(['id', 'moment','day', 'dagverloop_algemeen'])['duur'].sum()/60
moment1 = mm.groupby(['id', 'moment', 'dagverloop_algemeen']).mean()
moment1 = pd.DataFrame(data=moment1)

In [601]:
moment1_wide = pd.pivot_table(moment1, index= ["id"], columns= ["moment", "dagverloop_algemeen"], values=["duur"])

In [602]:
moment1_wide.columns = [("gem.min/dag", "WEEK1", "les1"), ("gem.min/dag", "WEEK1", "pauze1"),("gem.min/dag", "WEEK1", "vóór/na1"),("gem.min/dag", "WEEKEND1", "weekend1")]

In [603]:
moment1_wide.head()

,"(gem.min/dag, WEEK1, les1)","(gem.min/dag, WEEK1, pauze1)","(gem.min/dag, WEEK1, vóór/na1)","(gem.min/dag, WEEKEND1, weekend1)"
id,,,,
09c20aee-6789-40f2-ab64-95bcd190663c,15.897888,1.333467,117.995554,157.522617
0def5422-9828-43d4-8aa9-fd949779ba71,22.042542,1.606313,131.155737,271.848392
12d6ca9f-b961-48cf-a41a-0f5824c2b444,9.579022,1.395667,55.472958,72.465717
1e6526cd-4609-4c7f-9cea-4f4c0b47c24a,20.538921,2.079487,170.897671,268.898275
225d33b4-b261-4b7d-a18c-ed1909680462,94.476400,0.840750,127.818217,163.626508


In [604]:
leeg = pd.merge(leeg, moment1_wide, on="id")

<ul> algemeen gemiddeld: specifiek verloop  </ul>

In [617]:
mm = df_week1.groupby(['id', 'moment','day', 'dagverloop'])['duur'].sum()/60
moment_specifiek1 = mm.groupby(['id', 'moment', 'dagverloop']).mean()
moment_specifiek1 = pd.DataFrame(data=moment_specifiek1)

In [618]:
moment1_specifiek_wide = pd.pivot_table(moment_specifiek1, index= ["id"], columns= ["moment", "dagverloop"], values=["duur"])

In [620]:
moment1_specifiek_wide.columns = [('gem.min/dag', 'WEEK1', 'lesuur 1'),
 ('gem.min/dag', 'WEEK1', 'lesuur 2'),
 ('gem.min/dag', 'WEEK1',  'lesuur 3'),
 ('gem.min/dag', 'WEEK1',  'lesuur 4'),
 ('gem.min/dag', 'WEEK1',  'lesuur 5'),
 ('gem.min/dag', 'WEEK1',  'lesuur 6'),
 ('gem.min/dag', 'WEEK1',  'lesuur 7'),
 ('gem.min/dag', 'WEEK1',  'lesuur 8'),
 ('gem.min/dag', 'WEEK1', 'middagpauze'),
 ('gem.min/dag', 'WEEK1',  'na'),
 ('gem.min/dag', 'WEEK1', 'speeltijd 1'),
 ('gem.min/dag', 'WEEK1',  'speeltijd 2'),
 ('gem.min/dag', 'WEEK1',  'vóór'),
 ('gem.min/dag', 'WEEK1', 'weekend')]

In [621]:
leeg = pd.merge(leeg, moment1_specifiek_wide, on="id")

#### multitasking activity 

<ul> ses_prop: algemeen, school, speeltijd, voor-na </ul>
<ul> procent_leeg: algemeen, school, speeltijd, voor-na </ul>
<ul> not_MT: algemeen, school, speeltijd, voor-na</ul>
<ul> switches: algemeen, school, speeltijd, voor-na </ul>

#### well-being

totale score per participant

### week 2:

#### smartphonegebruik

In [622]:
#alles op basis van week  
df_week2 = df_adjust[df_adjust["beleid"] == "lesweek 2"]

<ul> algemeen gemiddeld dagelijks </ul>

In [623]:
mm = df_week2.groupby(['id', 'moment','day'])['duur'].sum()/60
algemeen2 = mm.groupby(['id']).mean()
algemeen2 = pd.DataFrame(data=algemeen2)

In [627]:
algemeen2.columns = ["alg_min_gemiddeld_dagelijks_W2"]

In [629]:
leeg = pd.merge(leeg, algemeen2, on="id")

<ul> algemeen gemiddeld tijdens weekdag & weekenddag </ul>

In [630]:
mm = df_week2.groupby(['id', 'moment','day'])['duur'].sum()/60
moment2 = mm.groupby(['id', 'moment']).mean()
moment2 = pd.DataFrame(data=moment2)

In [631]:
moment2_wide = pd.pivot_table(moment2, index= ["id"], columns= ["moment"], values=["duur"])

In [632]:
moment2_wide.columns = [("gem.min/dag", "WEEK2"), ("gem.min/dag", "WEEKEND2")]

In [633]:
moment2_wide

,"(gem.min/dag, WEEK2)","(gem.min/dag, WEEKEND2)"
id,,
09c20aee-6789-40f2-ab64-95bcd190663c,203.603979,189.011650
0def5422-9828-43d4-8aa9-fd949779ba71,179.791588,222.157806
12d6ca9f-b961-48cf-a41a-0f5824c2b444,156.871433,95.496061
1e6526cd-4609-4c7f-9cea-4f4c0b47c24a,214.348112,349.763750
225d33b4-b261-4b7d-a18c-ed1909680462,NaN,144.124133
9c2652af-446c-4442-afd4-c3e233392c34,175.987946,NaN
a29814bf-bd90-48e6-b42e-368285fd7579,125.311342,132.973800
bf6f10ed-d319-4309-9255-4ec60b3903ba,145.811367,NaN
c56b5c62-68e6-4cec-b30a-27defd151e2a,324.814921,456.781472


In [634]:
leeg = pd.merge(leeg, moment2_wide, on="id")

<ul> algemeen gemiddeld tijdens school: per dag vd week </ul>

In [635]:
mm = df_week2.groupby(['id', 'moment','day', 'dagverloop_algemeen'])['duur'].sum()/60
moment2 = mm.groupby(['id', 'moment', 'dagverloop_algemeen']).mean()
moment2 = pd.DataFrame(data=moment2)

In [636]:
moment2_wide = pd.pivot_table(moment2, index= ["id"], columns= ["moment", "dagverloop_algemeen"], values=["duur"])

In [518]:
moment2_wide.head()

duur                        \
moment                                     WEEK                         
dagverloop_algemeen                         les     pauze     vóór/na   
id                                                                      
09c20aee-6789-40f2-ab64-95bcd190663c  35.477313  2.636504  165.086975   
0def5422-9828-43d4-8aa9-fd949779ba71  15.112238  4.379129  160.300221   
12d6ca9f-b961-48cf-a41a-0f5824c2b444  19.713017  6.286858  134.014988   
1e6526cd-4609-4c7f-9cea-4f4c0b47c24a  19.208158  1.855825  193.284129   
225d33b4-b261-4b7d-a18c-ed1909680462        NaN       NaN         NaN   

                                                  
moment                                   WEEKEND  
dagverloop_algemeen                      weekend  
id                                                
09c20aee-6789-40f2-ab64-95bcd190663c  189.011650  
0def5422-9828-43d4-8aa9-fd949779ba71  222.157806  
12d6ca9f-b961-48cf-a41a-0f5824c2b444   95.496061  
1e6526cd-4609-4c7f-9cea-4f4c0b47c24a  349.763750  
225d33b4-b261-4b7d-a18c-ed1909680462  144.124133

In [637]:
moment2_wide.columns = [("gem.min/dag", "WEEK2", "les2"), ("gem.min/dag", "WEEK2", "pauze2"),("gem.min/dag", "WEEK2", "vóór/na2"),("gem.min/dag", "WEEKEND2", "weekend2")]

In [638]:
moment2_wide.head()

,"(gem.min/dag, WEEK2, les2)","(gem.min/dag, WEEK2, pauze2)","(gem.min/dag, WEEK2, vóór/na2)","(gem.min/dag, WEEKEND2, weekend2)"
id,,,,
09c20aee-6789-40f2-ab64-95bcd190663c,35.477313,2.636504,165.086975,189.011650
0def5422-9828-43d4-8aa9-fd949779ba71,15.112238,4.379129,160.300221,222.157806
12d6ca9f-b961-48cf-a41a-0f5824c2b444,19.713017,6.286858,134.014988,95.496061
1e6526cd-4609-4c7f-9cea-4f4c0b47c24a,19.208158,1.855825,193.284129,349.763750
225d33b4-b261-4b7d-a18c-ed1909680462,NaN,NaN,NaN,144.124133


In [640]:
leeg = pd.merge(leeg, moment2_wide, on = "id")

<ul> algemeen gemiddeld: specifiek verloop  </ul>

In [644]:
mm = df_week2.groupby(['id', 'moment','day', 'dagverloop'])['duur'].sum()/60
moment_specifiek2 = mm.groupby(['id', 'moment', 'dagverloop']).mean()
moment_specifiek2 = pd.DataFrame(data=moment_specifiek2)

In [645]:
moment2_specifiek_wide = pd.pivot_table(moment_specifiek2, index= ["id"], columns= ["moment", "dagverloop"], values=["duur"])

In [646]:
moment1_specifiek_wide.columns = [('gem.min/dag', 'WEEK2', 'lesuur 1'),
 ('gem.min/dag', 'WEEK2', 'lesuur 2'),
 ('gem.min/dag', 'WEEK2',  'lesuur 3'),
 ('gem.min/dag', 'WEEK2',  'lesuur 4'),
 ('gem.min/dag', 'WEEK2',  'lesuur 5'),
 ('gem.min/dag', 'WEEK2',  'lesuur 6'),
 ('gem.min/dag', 'WEEK2',  'lesuur 7'),
 ('gem.min/dag', 'WEEK2',  'lesuur 8'),
 ('gem.min/dag', 'WEEK2', 'middagpauze'),
 ('gem.min/dag', 'WEEK2',  'na'),
 ('gem.min/dag', 'WEEK2', 'speeltijd 1'),
 ('gem.min/dag', 'WEEK2',  'speeltijd 2'),
 ('gem.min/dag', 'WEEK2',  'vóór'),
 ('gem.min/dag', 'WEEK2', 'weekend')]

In [647]:
leeg = pd.merge(leeg, moment2_specifiek_wide, on="id")

C:\Users\beheerder\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:544: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 3 on the right)
  warnings.warn(msg, UserWarning)


In [648]:
leeg.head()

,id,log_dag,perdag_sec_over3wkn,perdag_min_over3wkn,min_WEEK_3wkn,min_WEEKEND_3wkn,"(gem_min_dag, WEEK1, lesweek1)","(gem_min_dag, WEEK1, lesweek2)","(gem_min_dag, WEEK3, lesweek3)","(gem_min_dag, WEEKEND1, lesweek1)",...,"(duur, WEEK, lesuur 5)","(duur, WEEK, lesuur 6)","(duur, WEEK, lesuur 7)","(duur, WEEK, lesuur 8)","(duur, WEEK, middagpauze)","(duur, WEEK, na)","(duur, WEEK, speeltijd 1)","(duur, WEEK, speeltijd 2)","(duur, WEEK, vóór)","(duur, WEEKEND, weekend)"
0,09c20aee-6789-40f2-ab64-95bcd190663c,18,10847.300389,180.788340,177.365427,176.416037,135.226908,203.603979,198.565383,157.522617,...,6.860983,25.652425,11.840908,6.051467,4.385917,147.746633,0.591394,NaN,17.340342,189.011650
1,0def5422-9828-43d4-8aa9-fd949779ba71,15,10894.386800,181.573113,163.412548,242.034040,154.804592,179.791588,132.328217,271.848392,...,2.790367,1.957900,2.189792,3.184458,4.606325,139.820183,2.000317,0.767639,20.480038,222.157806
2,12d6ca9f-b961-48cf-a41a-0f5824c2b444,17,5725.379059,95.422984,100.673555,86.283923,63.371875,156.871433,75.478622,72.465717,...,3.806900,NaN,NaN,0.584383,4.606650,120.697363,3.360417,NaN,17.756833,95.496061
3,1e6526cd-4609-4c7f-9cea-4f4c0b47c24a,15,13572.339400,226.205657,188.195076,317.417560,193.540092,214.348112,62.202867,268.898275,...,0.534608,NaN,1.106233,31.882967,1.840100,176.885979,0.752242,0.398517,16.398150,349.763750
4,225d33b4-b261-4b7d-a18c-ed1909680462,11,8995.026636,149.917111,152.741942,157.125717,159.870850,NaN,145.613033,163.626508,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.124133


leeg is opgeslagen als MobileDNA variabelen van Kortrijk. 

In [649]:
leeg.to_excel("study2_Kortrijk_MobileDNA.xlsx")

#### multitasking activity 

<ul> ses_prop: algemeen, school, speeltijd, voor-na </ul>
<ul> procent_leeg: algemeen, school, speeltijd, voor-na </ul>
<ul> not_MT: algemeen, school, speeltijd, voor-na</ul>
<ul> switches: algemeen, school, speeltijd, voor-na </ul>

#### well-being

totale score per participant

In [414]:
mm = df_adjust.groupby(['id','beleid','moment','day', 'dagverloop'])['duur'].sum()/60
smartphone = mm.groupby(['id', 'moment', 'beleid', 'dagverloop']).mean()
smartphone = pd.DataFrame(data=smartphone)

In [416]:
smartphone_wide = pd.pivot_table(smartphone, index= ["id"], columns= ["moment","beleid", "dagverloop"], values=["duur"])

In [417]:
smartphone_wide

duur                      \
moment                                     WEEK                       
beleid                                lesweek 1                       
dagverloop                             lesuur 1  lesuur 2 lesuur 2    
id                                                                    
09c20aee-6789-40f2-ab64-95bcd190663c   0.270450  4.227058       NaN   
0def5422-9828-43d4-8aa9-fd949779ba71        NaN  0.011600       NaN   
12d6ca9f-b961-48cf-a41a-0f5824c2b444   0.014383  1.976017  0.017417   
1e6526cd-4609-4c7f-9cea-4f4c0b47c24a   4.118167  1.545556       NaN   
225d33b4-b261-4b7d-a18c-ed1909680462        NaN  0.306567       NaN   
9c2652af-446c-4442-afd4-c3e233392c34  11.931567  8.817008  2.514933   
a29814bf-bd90-48e6-b42e-368285fd7579   2.541267  0.569183       NaN   
bf6f10ed-d319-4309-9255-4ec60b3903ba        NaN       NaN       NaN   
c56b5c62-68e6-4cec-b30a-27defd151e2a   2.394878  3.364842  0.629467   
ef622f43-8a33-47de-abf8-6d9b0be0d9aa   3.206558  0.194672       NaN   
f7e0880f-8471-4b1f-9b44-b2aff6263a0e   4.974900  2.481528       NaN   
f9a43cfe-6224-4b25-a46e-92341be7545b   5.050389  1.826950       NaN   

                                                                      \
moment                                                                 
beleid                                                                 
dagverloop                             lesuur 3   lesuur 4  lesuur 5   
id                                                                     
09c20aee-6789-40f2-ab64-95bcd190663c  18.506583   2.518967  0.029283   
0def5422-9828-43d4-8aa9-fd949779ba71  21.756283   0.642250  0.685267   
12d6ca9f-b961-48cf-a41a-0f5824c2b444   0.020467   1.294317       NaN   
1e6526cd-4609-4c7f-9cea-4f4c0b47c24a  17.738625   1.616800  1.737867   
225d33b4-b261-4b7d-a18c-ed1909680462        NaN   0.608867       NaN   
9c2652af-446c-4442-afd4-c3e233392c34   9.144308  16.894350  0.084517   
a29814bf-bd90-48e6-b42e-368285fd7579   1.779208   4.775817  0.048550   
bf6f10ed-d319-4309-9255-4ec60b3903ba        NaN        NaN       NaN   
c56b5c62-68e6-4cec-b30a-27defd151e2a   4.202488   1.629225  2.904439   
ef622f43-8a33-47de-abf8-6d9b0be0d9aa   9.232750   1.592033  9.978817   
f7e0880f-8471-4b1f-9b44-b2aff6263a0e  11.701612   7.674711  0.024017   
f9a43cfe-6224-4b25-a46e-92341be7545b   9.000422   3.006917  0.647583   

                                                                      \
moment                                                                 
beleid                                                                 
dagverloop                            lesuur 6   lesuur 7   lesuur 8   
id                                                                     
09c20aee-6789-40f2-ab64-95bcd190663c  2.652744   1.807267   0.208233   
0def5422-9828-43d4-8aa9-fd949779ba71  1.147678   7.995872  11.359594   
12d6ca9f-b961-48cf-a41a-0f5824c2b444  0.088625  13.636783   7.731033   
1e6526cd-4609-4c7f-9cea-4f4c0b47c24a  0.606750   7.219325   3.173072   
225d33b4-b261-4b7d-a18c-ed1909680462       NaN  47.538400  46.022567   
9c2652af-446c-4442-afd4-c3e233392c34  4.483475  14.672150   8.159622   
a29814bf-bd90-48e6-b42e-368285fd7579  0.635467  34.983367  17.049528   
bf6f10ed-d319-4309-9255-4ec60b3903ba       NaN        NaN        NaN   
c56b5c62-68e6-4cec-b30a-27defd151e2a  4.365333   4.062567   6.677383   
ef622f43-8a33-47de-abf8-6d9b0be0d9aa  4.504567        NaN        NaN   
f7e0880f-8471-4b1f-9b44-b2aff6263a0e  3.728342   0.428183   1.601167   
f9a43cfe-6224-4b25-a46e-92341be7545b  6.479992  16.564833   6.652350   

                                                          ...                \
moment                                                    ...                 
beleid                                                    ...     lesweek 3   
dagverloop                           middag speeltijd     ...      lesuur 6   
id                                                        ...        

## Berekenen: verschil in smartphonegebruik tijdens school => fact check 

effectiviteit van smartphoneban? positief verschil in smartphonegebruik in school. 

De juiste variabelen hiervoor selecteren => nodig: moment = WEEK, beleid = lesweek 1 & lesweek 2, dagverloop_algemeen = les & pauze 

In [430]:
df_FC = df_adjust[df_adjust["moment"] == "WEEK"]

In [431]:
df_FC = df_FC[df_FC["beleid"] != "lesweek 3"]

In [434]:
df_FC = df_FC[df_FC["dagverloop_algemeen"] != "vóór/na"]

In [435]:
df_FC = df_FC[df_FC["dagverloop_algemeen"] != "weekend"]

In [445]:
mm = df_FC.groupby(['id','beleid', 'day', 'dagverloop_algemeen'])['duur'].sum()/60
factcheck = mm.groupby(['id','beleid','dagverloop_algemeen']).mean()
factcheck = pd.DataFrame(data=factcheck)

In [446]:
verschil in gebruik tijdens de les

SyntaxError: invalid syntax (<ipython-input-446-c51f9f7e58d0>, line 1)

In [448]:
factcheck = factcheck.reset_index()

In [449]:
factcheck_wide = pd.pivot_table(factcheck, index= ["id"], columns= ["beleid", "dagverloop_algemeen"], values=["duur"])

In [450]:
factcheck_wide

duur                        \
beleid                                lesweek 1             lesweek 2   
dagverloop_algemeen                         les      pauze        les   
id                                                                      
09c20aee-6789-40f2-ab64-95bcd190663c  15.897888   1.333467  35.477313   
0def5422-9828-43d4-8aa9-fd949779ba71  22.042542   1.606313  15.112238   
12d6ca9f-b961-48cf-a41a-0f5824c2b444   9.579022   1.395667  19.713017   
1e6526cd-4609-4c7f-9cea-4f4c0b47c24a  20.538921   2.079487  19.208158   
225d33b4-b261-4b7d-a18c-ed1909680462  94.476400   0.840750        NaN   
9c2652af-446c-4442-afd4-c3e233392c34  29.213808   2.643517   5.513587   
a29814bf-bd90-48e6-b42e-368285fd7579  27.262821   1.311806  12.789971   
c56b5c62-68e6-4cec-b30a-27defd151e2a  25.052471   6.615163  37.838467   
ef622f43-8a33-47de-abf8-6d9b0be0d9aa  11.476696  12.496967   9.035604   
f7e0880f-8471-4b1f-9b44-b2aff6263a0e  26.679512  14.821333  30.377683   
f9a43cfe-6224-4b25-a46e-92341be7545b  27.803083   3.374142  30.082862   

                                                
beleid                                          
dagverloop_algemeen                      pauze  
id                                              
09c20aee-6789-40f2-ab64-95bcd190663c  2.636504  
0def5422-9828-43d4-8aa9-fd949779ba71  4.379129  
12d6ca9f-b961-48cf-a41a-0f5824c2b444  6.286858  
1e6526cd-4609-4c7f-9cea-4f4c0b47c24a  1.855825  
225d33b4-b261-4b7d-a18c-ed1909680462       NaN  
9c2652af-446c-4442-afd4-c3e233392c34  1.889533  
a29814bf-bd90-48e6-b42e-368285fd7579  2.283083  
c56b5c62-68e6-4cec-b30a-27defd151e2a  5.810321  
ef622f43-8a33-47de-abf8-6d9b0be0d9aa  3.396339  
f7e0880f-8471-4b1f-9b44-b2aff6263a0e  8.697667  
f9a43cfe-6224-4b25-a46e-92341be7545b  1.393029

In [451]:
list(factcheck_wide)

[('duur', 'lesweek 1', 'les'),
 ('duur', 'lesweek 1', 'pauze'),
 ('duur', 'lesweek 2', 'les'),
 ('duur', 'lesweek 2', 'pauze')]

In [452]:
factcheck_wide[('duur', 'lesweek_verschil', 'les')] = factcheck_wide[('duur', 'lesweek 1', 'les')] - factcheck_wide[('duur', 'lesweek 2', 'les')]

In [453]:
factcheck_wide[('duur', 'lesweek_verschil', 'pauze')] = factcheck_wide[('duur', 'lesweek 1', 'pauze')] - factcheck_wide[('duur', 'lesweek 2', 'pauze')]

In [454]:
factcheck_wide

duur                        \
beleid                                lesweek 1             lesweek 2   
dagverloop_algemeen                         les      pauze        les   
id                                                                      
09c20aee-6789-40f2-ab64-95bcd190663c  15.897888   1.333467  35.477313   
0def5422-9828-43d4-8aa9-fd949779ba71  22.042542   1.606313  15.112238   
12d6ca9f-b961-48cf-a41a-0f5824c2b444   9.579022   1.395667  19.713017   
1e6526cd-4609-4c7f-9cea-4f4c0b47c24a  20.538921   2.079487  19.208158   
225d33b4-b261-4b7d-a18c-ed1909680462  94.476400   0.840750        NaN   
9c2652af-446c-4442-afd4-c3e233392c34  29.213808   2.643517   5.513587   
a29814bf-bd90-48e6-b42e-368285fd7579  27.262821   1.311806  12.789971   
c56b5c62-68e6-4cec-b30a-27defd151e2a  25.052471   6.615163  37.838467   
ef622f43-8a33-47de-abf8-6d9b0be0d9aa  11.476696  12.496967   9.035604   
f7e0880f-8471-4b1f-9b44-b2aff6263a0e  26.679512  14.821333  30.377683   
f9a43cfe-6224-4b25-a46e-92341be7545b  27.803083   3.374142  30.082862   

                                                                           
beleid                                         lesweek_verschil            
dagverloop_algemeen                      pauze              les     pauze  
id                                                                         
09c20aee-6789-40f2-ab64-95bcd190663c  2.636504       -19.579425 -1.303038  
0def5422-9828-43d4-8aa9-fd949779ba71  4.379129         6.930304 -2.772817  
12d6ca9f-b961-48cf-a41a-0f5824c2b444  6.286858       -10.133994 -4.891192  
1e6526cd-4609-4c7f-9cea-4f4c0b47c24a  1.855825         1.330762  0.223662  
225d33b4-b261-4b7d-a18c-ed1909680462       NaN              NaN       NaN  
9c2652af-446c-4442-afd4-c3e233392c34  1.889533        23.700221  0.753983  
a29814bf-bd90-48e6-b42e-368285fd7579  2.283083        14.472850 -0.971278  
c56b5c62-68e6-4cec-b30a-27defd151e2a  5.810321       -12.785996  0.804842  
ef622f43-8a33-47de-abf8-6d9b0be0d9aa  3.396339         2.441092  9.100628  
f7e0880f-8471-4b1f-9b44-b2aff6263a0e  8.697667        -3.698171  6.123667  
f9a43cfe-6224-4b25-a46e-92341be7545b  1.393029        -2.279779  1.981113

In [ ]:
=> belangrijk dat deze namen nog worden aangepast en daarna ook in de leeg worden opgenomen. 

In [455]:
factcheck_wide.to_excel("factcheck.xlsx")

In [465]:
factcheck_wide.describe()

duur                                   \
beleid               lesweek 1             lesweek 2              
dagverloop_algemeen        les      pauze        les      pauze   
count                11.000000  11.000000  10.000000  10.000000   
mean                 28.183924   4.410783  21.514890   3.862829   
std                  22.980859   4.868600  11.307951   2.393273   
min                   9.579022   0.840750   5.513587   1.393029   
25%                  18.218404   1.364567  13.370538   1.987921   
50%                  25.052471   2.079487  19.460588   3.016422   
75%                  27.532952   4.994652  30.303978   5.452523   
max                  94.476400  14.821333  37.838467   8.697667   

                                                 
beleid              lesweek_verschil             
dagverloop_algemeen              les      pauze  
count                      10.000000  10.000000  
mean                        0.039786   0.904957  
std                        12.889765   4.103872  
min                       -19.579425  -4.891192  
25%                        -8.525039  -1.220098  
50%                        -0.474508   0.488823  
75%                         5.808001   1.687045  
max                        23.700221   9.100628

###### gemiddeld dagelijks gebruik tijdens les week 1 - gemiddeld dagelijks gebruik tijdens les week 2 = positief? 

In [ ]:
kan dan ook onderzocht worden in multitasking activity? Nuttig of over the top? 

## Berekenen: verschil in smartphone gebruik tijdens speeltijd 

Heeft de smartphoneban een effect op smartphonegebruik tijdens speeltijd? en op within smartphone multitasking? 

###### gemiddeld dagelijks gebruik tijdens speeltijd week 1 - gemiddeld dagelijks gebruik tijdens speeltijd week 2 = ? 

In [ ]:
Wordt hierboven ook al weergegeven. 

## Berekenen: verschil in smartphone gebruik voor/na school 

Heeft de smartphoneban een effect op smartphonegebruik na school? compensatiedrang? invloed op within smartphone multitasking? 

###### gemiddeld dagelijks gebruik voor/na schooltijd week 1 - gemiddeld dagelijks gebruik voor/na schooltijd week 2 = ? 

In [457]:
df_VN = df_adjust[df_adjust["moment"] == "WEEK"]
df_VN = df_VN[df_VN["beleid"] != "lesweek 3"]
df_VN = df_VN[df_VN["dagverloop_algemeen"] == "vóór/na"]

In [458]:
mm = df_VN.groupby(['id','beleid', 'day', 'dagverloop_algemeen'])['duur'].sum()/60
compensatie = mm.groupby(['id','beleid','dagverloop_algemeen']).mean()
compensatie = pd.DataFrame(data=compensatie)

In [459]:
compensatie

duur
id                                   beleid    dagverloop_algemeen            
09c20aee-6789-40f2-ab64-95bcd190663c lesweek 1 vóór/na              117.995554
                                     lesweek 2 vóór/na              165.086975
0def5422-9828-43d4-8aa9-fd949779ba71 lesweek 1 vóór/na              131.155737
                                     lesweek 2 vóór/na              160.300221
12d6ca9f-b961-48cf-a41a-0f5824c2b444 lesweek 1 vóór/na               55.472958
                                     lesweek 2 vóór/na              134.014988
1e6526cd-4609-4c7f-9cea-4f4c0b47c24a lesweek 1 vóór/na              170.897671
                                     lesweek 2 vóór/na              193.284129
225d33b4-b261-4b7d-a18c-ed1909680462 lesweek 1 vóór/na              127.818217
9c2652af-446c-4442-afd4-c3e233392c34 lesweek 1 vóór/na               99.194662
                                     lesweek 2 vóór/na              169.529592
a29814bf-bd90-48e6-b42e-368285fd7579 lesweek 1 vóór/na              135.947496
                                     lesweek 2 vóór/na              110.809058
bf6f10ed-d319-4309-9255-4ec60b3903ba lesweek 2 vóór/na              145.811367
c56b5c62-68e6-4cec-b30a-27defd151e2a lesweek 1 vóór/na              217.308233
                                     lesweek 2 vóór/na              281.052025
ef622f43-8a33-47de-abf8-6d9b0be0d9aa lesweek 1 vóór/na               66.680825
                                     lesweek 2 vóór/na               89.859046
f7e0880f-8471-4b1f-9b44-b2aff6263a0e lesweek 1 vóór/na              236.363646
                                     lesweek 2 vóór/na              158.616983
f9a43cfe-6224-4b25-a46e-92341be7545b lesweek 1 vóór/na              227.254579
                                     lesweek 2 vóór/na              212.708754

In [460]:
compensatie_wide = pd.pivot_table(compensatie, index= ["id"], columns= ["beleid", "dagverloop_algemeen"], values=["duur"])

In [462]:
compensatie_wide[('duur', 'lesweek_verschil', 'vóór/na')] = compensatie_wide[('duur', 'lesweek 1', 'vóór/na')] - compensatie_wide[('duur', 'lesweek 2', 'vóór/na')]

In [463]:
compensatie_wide.describe()

duur                             
beleid                lesweek 1   lesweek 2 lesweek_verschil
dagverloop_algemeen     vóór/na     vóór/na          vóór/na
count                 11.000000   11.000000        10.000000
mean                 144.189962  165.552103       -21.699041
std                   62.088858   51.512414        48.835429
min                   55.472958   89.859046       -78.542029
25%                  108.595108  139.913177       -59.580699
50%                  131.155737  160.300221       -26.161352
75%                  194.102952  181.406860         5.312754
max                  236.363646  281.052025        77.746663

In [ ]:
negatieve verschil - waarde = meer compensatiegedrag. 

In [ ]:
Dit kan nu ook weggeschreven worden en moet ook in de lege dataframe worden geplaatst. 

In [464]:
compensatie_wide.to_excel("compensatie_wide.xlsx")

In [466]:
leeg.to_excel("leeg.xlsx")

## Berekenen: verschil in welzijn 

###### welzijn week 1 -  welzijn week 2 = ? 

In [115]:
dfBr.to_excel("Bruggeweek1.xlsx")